In [ ]:
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, windows
from scipy.stats import linregress
%matplotlib inline
import scipy.io
from scipy.signal import resample, freqz
from scipy.interpolate import interp1d
from scipy.signal import remez, firwin, lfilter
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import PCA
import random
import matplotlib.cm as cm
import os
import pandas as pd
from scipy.signal import csd, detrend
import pickle
from scipy.fft import fft, fftfreq
import sys
import scipy.ndimage

In [ ]:
### PARAMETERS ###
start_scope() # Re-initialize Brian
plt.style.use('_classic_test_patch') # Plotting style

### GENERAL PARAMETERS ###########################################################
sim_name = 'PAIN_sim_testing_no_inhib' #'PAIN_sim_100percent_exponential_distrib_inhib_test'

# TIME PARAMETERS
fsamp = 1000  # set your fsamp # This is NOT the dt at which the simulation runs. The simulation timesteps are 0.1ms in duration by default
window_beginning_ignore = 1 # in s
window_end_ignore = 1 # in s
ISI_threshold_for_discontinuity = 0.4 # in s ; motoneurons whose max(ISI)>threshold will be removed from analysis (so only continuous MNs are kept)

# VOLTAGE THRESHOLDS OF ALL NEURONS
voltage_rest = 0 # arbitrary ; 0 at rest
voltage_thresh = 1 # arbitrary ; 1 for generating a spike

# NUMBER OF NEURONS SIMULATED
nb_motoneurons_full_pool = 300 # All motoneurons from the pool to be simulated

# SIMULATING THE HD-EMG MU IDENTIFICATION PROCESS BY SUB-SAMPLING THE ACTIVE MUs
subsample_MUs_for_analysis = False # True
nb_of_MUs_to_subsample = 50
motor_unit_subsampling_probability_distribution = 'size' # 'size' #'uniform', 'size' # if 'uniform", every active MU will have the same probability to be selected for the analysis; if 'size", larger motor units will have a higher probability of being selected
bias_towards_larger_motor_neurons_temperature = 5.0 # only used if motor_unit_subsampling_probability_distribution=='size'.
# If infinity (inf), same as uniform distribution. If ~100, probability is approximately linearly scaled according to size. Exponential bias for values below (very strong bias for temperature = 1 for example). Error if 0


### TARGET SIMULATION
target_type = 'trapezoid' #'sinusoid' # 'plateau' #'trapezoid'
target_force_level = 30 # % of the max tetanic force of the simulated MNs
# if trapezoid:
ramp_duration = 6 # in s
plateau_duration = 60 # 60 # 20 # in s
analyzis_window = 'plateau'# 'all' 'plateau' # if 'all', the entire signal will be analyzed. If 'plateau', only the plateau section will be analyzed.
# if sinusoid:
target_force_sin_freq = 1 # used only if targt_type == 'sinusoid'

### SIMULATION DURATION
if target_type != 'trapezoid':
    true_duration = 20
else:
    true_duration = ramp_duration*2 + plateau_duration
duration_with_ignored_window = (true_duration+window_beginning_ignore+window_end_ignore)*second


### INPUT PARAMETERS #########################################################

# INHIBITORY INPUT
inhibition_distribution = 'homogeneous' # 'heterogeneous', 'homogeneous'
# 'heterogeneous' = Distribution randomly distributed to increasing X% of motor neurons
# 'homogeneous' = Identical distribution of inhibition to all motor neurons (equivalent to 'heterogeneous' with 'proportion_of_MN_affected_by_each_inhibitory_input = 100')
inhibition_weight_distribution = 'uniform' #'uniform" 'exponential'
# 'exponential' = MNs receiving inhibition will receive increasing or decreasing amount of inhibition according to their sizes - Martinez Valdes J physiol 2020 model = https://physoc.onlinelibrary.wiley.com/doi/full/10.1113/JP279225 
# 'uniform' = all MNs receiving inhibition will receive the same amount of inhibition (identical to 'exponential' with 'inhibition_exponent_weights = 0')
inhibition_exponent_weights_original = 20 # very high but allows to get a meaningful difference in the simulated and active MNs # used only if 'inhibition_weight_distribution = exponential'
inhibition_constant_weights_original = 1 # used only if 'inhibition_weight_distribution = exponential'
inhibition_offset_weights_original = 0
# Distribution of inhibitory weights = constant * MN_size^(exponent) + offset
proportion_of_MN_affected_by_each_inhibitory_input_original = 100 # 44 # in % (used only if 'inhibition_distribution = heterogeneous')
# ^ given the current parameters, the mean of the weights for the distribution of inhibitory input is ~44% when the distribution is exponential and the proportion of MN affected = 100 (when sampling all continuous motor units), so using 44 when distributing inhibition randomly among motor units
nb_inhibitory_input = 0
low_pass_filter_of_inhibitory_input = 5 #in hz
inhibitory_input_mean = -0.5 #-0.5 #-0.1 -0.05 # if inhib_before_force_optimization = False, inhibitory_input_mean should be close to 0 (otherwise very few MUs will fire and the force will be low)
# use -0.5 if inhib_before_force_optimization = True; use -0.05 if inhib_before_force_optimization = False
inhibitory_input_std = 0.025
prevent_inhibition_from_being_positive = True # If the inhibition is noisy (std != 0), this makes sure that the inhiitory input will alway be < 0 (inhibitory)
# Determine in which the inhibition will happen in the script
inhib_before_force_optimization = True # True # False # If True, the force will be optimized taking the inhibitory input into account, so the common excitatory input will necessarily increase
# If false, the inhibitory input will be delivered after the common excitatory input has been optimized to reach the target force. So the force level will be reduced, but the common excitatory input will remain the same as when there is no inhibition.

inhibitory_input_source = 'generate_synthetic_input' # 'generate_synthetic_input' ; 'load_synthetic_input'
inhibitory_input_sourcefile = 'D:/THESE/Git_Scripts/Python_Scripts/motoneuron_simulation/Synthetic_signals.csv' # used only if inhibitory_input_source == 'load_synthetic_input'
# use the N first signals (the first N columns) as inhibitory inputs, with N = nb_inhibitory_input
# Use a .csv file. The number of samples in the csv file should be >= the number of samples in the simulation
inhibitory_input_sourcefile_fsamp = 1000 # 1000 if .csv ; 2048 if .mat # If not matching the simulation's fsamp, the loaded signal will be upsampled or downsampled to match the simulation's sampling rate


# COMMON EXCITATORY INPUT
common_input_baseline = voltage_thresh + 1.5
common_input_fluctuations_amplitude = 0.025 # 0.025 # Added to the "baseline common input" learned by optimization (reducing error between force produced and target force)
low_pass_filter_of_common_noise = 5 # in hz

common_input_source = 'generate_synthetic_input' # 'generate_synthetic_input' ; 'load_synthetic_input' ; 'load_experimental_data'
common_input_sourcefile = 'D:/THESE/Git_Scripts/Python_Scripts/motoneuron_simulation/Synthetic_signals.csv' # used only if common_input_source == 'load_synthetic_input' or 'load_experimental_data'
# Use the last signal (last column) as common noise
# If 'load_gaussian_noise', use a .csv file. The number of samples in the csv file should be >= the number of samples in the simulation
# If 'load_experimental_data', use one .mat file or several .mat files. If several .mat files are selected, they will be concatenated. The number of samples in the (concatenated) file(s) should be >= the number of samples in the simulation
common_input_sourcefile_fsamp = 1000 # 1000 if .csv ; 2048 if .mat # If not matching the simulation's fsamp, the loaded signal will be upsampled or downsampled to match the simulation's sampling rate

# INDEPENDENT NOISE
low_pass_filter_of_independent_noise = 50 # in hz
independent_noise_amplitude = 0.05 # common_input_fluctuations_amplitude * 2 # the value of noise in std (mean of 0)


### MOTONEURON PROPERTIES ACCORDING TO THEIR SIZES ##########################

# Electrophysiological properties calculated from Caillet et al 2022 https://elifesciences.org/articles/76489
# Assuming that soma diameter from human motoneurons vary between 50 and 100 micrometers, based on https://journals.physiology.org/doi/full/10.1152/physiol.00021.2018
    # ^ "Scaling of motoneurons, From Mouse to Human" Manuel et al. Physiology (2018)
min_soma_diameter = 50 # in micrometers, for smallest MN
max_soma_diameter = 100 # in micrometers, for largest MN
# Select the range of MNs simulates (0 being smallest, 100 being largest)
min_normalized_boundary = 0 #0 # Normalized between 0 and 100
max_normalized_boundary = 100 # Normalized between 0 and 100 - use a value below 100 if not simulating very fast MNs #20 to go up to 20% MVC

# DISTRIBUTION OF MOTONEURON SIZES
# Parameter to create an exponetially decreasing dsitribution curve, with larger motoneurons being less numerous than smaller motoneurons
# Somewhat fitting the curve in Principles of Neural Science 2021 edition, Enoka chapter on motor units, fig 31-3.A
size_distribution_exponent = 2
# size_distribution_exponent between 0.8 and 0.9 results in a quasi-uniform distribution of MN sizes

#### ELECTROPHYSIOLOGICAL MN PROPERTIES #####
tau_constant = 2.6*(10**4) # Caillet et al 2022
tau_exponent = 1.5 # Caillet et al 2022
    # https://www.desmos.com/calculator/bfhcpgiltr = visualize the curve
    # Min time constant for smallest MN (50 micrometers) = ~26ms
    # Max time constant for biggest MN (100 micrometers) = ~70ms
    # From Williams & Baker 2009 = "These decay times correspond with exponential time constants of 24 –26 ms, in keeping with previous models of motoneurons (Matthews, 1997)"
    # From Vertebrate Motoneurons 2022:
        # 3-15ms in cat ; 2-20ms in mouse
        # "Consequently, several labs use the half-decay time of AHP to distinguish between F-fast and S-slow rat motoneurons (F > 20 ms, S < 20 ms, (Gardiner 1993)) but the difference between the FR and FF motoneurons is not well defned."
    # From "Principles of Neural Science":
        # "Typical values of τ for neurons range from 20 to 50ms" (but not motor neurons specifically)
    # From the same book: "Cell membrane time constant; the product of resistance and capacitance of the membrane (typical values 1–20 ms). tau = Rm ⋅ Cm"
    # Maltenfort, Heckman 1998 simulation study = 2.5-10ms time constant for motoneurons
#### Refractory period, linear relationship (Caillet's paper gives equations for AHP duration but not for refractory period duration) #####
refractory_period_smallest_MN = 20*ms
refractory_period_largest_MN = 5*ms
    # Manuel et al. 2019 "Scaling of motor output, from Mouse to Humans"
        # "Statistical methods employed at low firing rates indicate the AHP durations of low-threshold human motoneurons, presumably type S and perhaps some type FR, are ~125–140 ms."
    # Herbert & Gandevia 1999 assume a 5ms (absolute?) refractory period
    # Lateva et at 2001 = Absolute refractory period of 3ms in muscle fibers, and relative refractory period of 10ms
    # University of Washington textbook of physiology = in a typical neuron, the absolute refractory period lasts a few ms and the relative period tens of ms
    # Tuned to avoid unrealistically high discharge rates when computing the MVC => 20ms maxes out the discharge rate of small MNs at 50pps
#### Input weight = normalizd resistance, so that the input to the smallest MN is scaled by a factor of 0 #####
resistance_constant = 9.6*(10**5) # Caillet et al 2022
resistance_exponent = 2.4 # Caillet et al 2022
    # https://www.desmos.com/calculator/pbs97zynff = visualize the curve for resistance (ohms) and input weights (between 0 and 1)
    # Min input weight for smallest MN (50 micrometers) = 1
    # Max input weight for biggest MN (100 micrometers) = ~0.19

### CONTRACTILE PROPERTIES (TWITCH FORCE CAUSED BY MN SPIKES)
    # Every firing of motor units will be convolved with a kernel
    #  => The kernel is a hanning window of duration which is 2x the time to peak force, and then the duration of the "down" portion of the twitch (when the force returns to baseline) is extended by 'multiplication_of_twitch_force_down_time'
    #   # Finally, the conduction delay is added before the start of the kernel
    # Linear interpolation to get force produced (this is a gross simplification, as the distribution of MU properties is not linearly distributed at all)
    # Data from Principles of Neural Science 2021 edition Motor Unit chapter Enoka # Figure 31-3, values for human tibialis anterior motor units => between 0-10 mN/m for smallest MUs, ~140mN/m for largest MUs
# Value in twitch torque (milliNewton/m)
twitch_force_range_small_MU = 5 # in milliNewton/meter
twitch_force_range_large_MU = 140 # in milliNewton/meter
    # The twitch force values are later normalized according to % of MVC, with the MVC being calculated as the force produces when all simulatd motor units receive extremely strong excitatory input
# twitch duration (Duration of the Gaussian kernel) => linear relationship (this is a gross simplification, as the distribution of MU properties is not linearly distributed at all)
    # ~20ms to peak force for fastest motor units ; ~80ms to peak force for slowest motor units - Principles of Neural Science 2021 edition Motor Unit chapter Enoka # Figure 31-3, values for human tibialis anterior motor units
    # Doubling the value because this is time to peak force, and kernel duration is twice that
    #    # Also some data from Shoepe et al 2003 MSSE, shortening velocity of type I fiber (twitch_duration small MU = 120ms, time to peak force = 60ms) VS type IIa fiber (twitch duration fast MU = 50ms, time to peak force = 25ms)
    #    # https://paulogentil.com/pdf/Functional%20adaptability%20of%20muscle%20fibers%20to%20long-term%20resistance%20exercise.pdf
time_to_peak_twitch_force_range_small_MU = 0.16 # in s
time_to_peak_twitch_force_range_large_MU = 0.04 # in s # biggest MU
multiplication_of_twitch_force_down_time = 4 # This is a gross simplificaion of how the twitch force return to baseline, but overall it seems to fit the behavior of motor units
    # B. R. Botterman, G. A. Iwamoto, and W. J. Gonyea (1986) = force trace of single twitchs from motor units
    # Rositsa Raikova, Piotr Krutki, Jan Celichowski (2023) => Detailed model of motor units twitch force
# Electromechanical delay => inverse of actional conduction velocity
axonal_conduction_velocity_constant = 4.0*2 # Inferred from the axonal conduction velocity relationship reported in Caillet et al 2022,
# ^ multiplying by two (assuming a 0.5m axon length => so correspond to the conduction speed from MN to muscle fiber)
axonal_conduction_velocity_exponent = 0.7 # Caillet et al 2022
low_pass_filter_force = False # Can be turned to true if fast oscillation (high frequency components) in force output
low_pass_filter_of_force_cutoff = 5 # in hz

### PARAMETERS FOR TESTING - reduce numbers for faster simulations
COH_calc_max_iteration_nb_per_group_size = 1000 # 10 for faster simulation
# More iteration for smaller group sizes, because the value obtained is very dependent upon the exact neurons selected, especially when only a few MNs are used to create the CST
# ^ the number of iteration will be "COH_calc_max_iteration_nb_per_group_size / nb_of_MUs_in_CST"
num_optimization_iterations = 10 # 5 for faster simulation


In [ ]:
### SAVE PARAMETERS
new_directory = sim_name
new_filename = 'parameters.txt'

# Create the directory if it doesn't exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)
else:
    directory_n = 0
    while os.path.exists(new_directory):
        directory_n = directory_n+1
        new_directory = str(sim_name + "_iter_" + str(directory_n))
        if not os.path.exists(new_directory):
            os.makedirs(new_directory)
            break
        if directory_n > 99: # prevent infinite loop
            break
save_file_path = os.path.join(new_directory, new_filename)

# Write the variables to the file
with open(save_file_path, 'w') as file:
    file.write(f"General parameters -----\n")
    file.write(f"   duration_with_ignored_window: {duration_with_ignored_window}\n")
    file.write(f"   nb_motoneurons_full_pool_per_pool: {nb_motoneurons_full_pool}\n")
    file.write(f"Sub-sampling of motor units (simulating the hdEMG motor unit identification process) -----\n")
    file.write(f"   subsample_MUs_for_analysis: {subsample_MUs_for_analysis}\n")
    file.write(f"   nb_of_MUs_to_subsample: {nb_of_MUs_to_subsample}\n")
    file.write(f"   motor_unit_subsampling_probability_distribution: {motor_unit_subsampling_probability_distribution}\n")
    file.write(f"   bias_towards_larger_motor_neurons_temperature: {bias_towards_larger_motor_neurons_temperature}\n")

    file.write(f"\n")
    file.write(f"Input parameters -----\n")
    file.write(f"   Common exitatory input -----\n")
    file.write(f"       common_input_baseline (before optimized input learning): {common_input_baseline}\n")
    file.write(f"       common_input_fluctuations_amplitude: {common_input_fluctuations_amplitude }\n")
    file.write(f"       common_input_source: {common_input_source }\n")
    file.write(f"       common_input_sourcefile: {common_input_sourcefile }\n")
    file.write(f"       common_input_sourcefile_fsamp: {common_input_sourcefile_fsamp }\n")
    file.write(f"   Independent input (noise) -----\n")
    file.write(f"       low_pass_filter_of_independent_noise: {low_pass_filter_of_independent_noise}\n")
    file.write(f"       independent_noise_amplitude: {independent_noise_amplitude}\n")
    file.write(f"   Inhibitory input (noise) -----\n")
    file.write(f"       inhibition_distribution: {inhibition_distribution}\n")
    file.write(f"       inhibition_weight_distribution: {inhibition_weight_distribution}\n")
    file.write(f"        - If exponential distribution of inhibitory input:\n")
    file.write(f"           inhibition_exponent_weights_original: {inhibition_exponent_weights_original}\n")
    file.write(f"           inhibition_constant_weights_original: {inhibition_constant_weights_original}\n")
    file.write(f"           inhibition_offset_weights_original: {inhibition_offset_weights_original}\n")
    file.write(f"        - If heterogeneous weight distribution of inhibitory input:\n")
    file.write(f"           proportion_of_MN_affected_by_each_inhibitory_input_original: {proportion_of_MN_affected_by_each_inhibitory_input_original}\n")
    file.write(f"       nb_inhibitory_input: {nb_inhibitory_input}\n")
    file.write(f"       low_pass_filter_of_inhibitory_input: {low_pass_filter_of_inhibitory_input}\n")
    file.write(f"       inhibitory_input_mean: {inhibitory_input_mean}\n")
    file.write(f"       inhibitory_input_std: {inhibitory_input_std}\n")
    file.write(f"       prevent_inhibition_from_being_positive: {prevent_inhibition_from_being_positive}\n")
    file.write(f"       inhibitory_input_source: {inhibitory_input_source}\n")
    file.write(f"       inhibitory_input_sourcefile: {inhibitory_input_sourcefile}\n")
    file.write(f"       inhibitory_input_sourcefile_fsamp: {inhibitory_input_sourcefile_fsamp}\n")

    file.write(f"\n")
    file.write(f"Force target parameters -----\n")
    file.write(f"   target_type: {target_type}\n")
    file.write(f"   target_force_level: {target_force_level}% MVC\n")
    file.write(f"   If trapezoid:\n")
    file.write(f"       ramp_duration: {ramp_duration}\n")
    file.write(f"       plateau_duration: {plateau_duration}\n")
    file.write(f"   If NOT trapezoid:\n")
    file.write(f"       true_duration: {true_duration}\n")
    file.write(f"   If sisnusoid:\n")
    file.write(f"       target_force_sin_freq (used only if targt_type == 'sinusoid'): {target_force_sin_freq}\n")
    file.write(f"   low_pass_filter_force: {low_pass_filter_force}\n")
    file.write(f"   low_pass_filter_of_force_cutoff: {low_pass_filter_of_force_cutoff}\n")

    file.write(f"Motor neurons size -----\n")
    file.write(f"   min_soma_diameter: {min_soma_diameter}\n")
    file.write(f"   max_soma_diameter: {max_soma_diameter}\n")
    file.write(f"   min_normalized_boundary : {min_normalized_boundary}\n")
    file.write(f"   max_normalized_boundary : {max_normalized_boundary}\n")
    file.write(f"   size_distribution_exponent : {size_distribution_exponent}\n")

    file.write(f"\n")
    file.write(f"Twitch force properties - parameters -----\n")
    file.write(f"   twitch_force_range_small_MU: {twitch_force_range_small_MU}\n")
    file.write(f"   twitch_force_range_large_MU: {twitch_force_range_large_MU}\n")
    file.write(f"   time_to_peak_twitch_force_range_small_MU: {time_to_peak_twitch_force_range_small_MU}\n")
    file.write(f"   time_to_peak_twitch_force_range_large_MU: {time_to_peak_twitch_force_range_large_MU}\n")
    file.write(f"   multiplication_of_twitch_force_down_time: {multiplication_of_twitch_force_down_time}\n")
    file.write(f"   axonal_conduction_velocity_constant: {axonal_conduction_velocity_constant}\n")
    file.write(f"   axonal_conduction_velocity_exponent: {axonal_conduction_velocity_exponent}\n")

    file.write(f"\n")
    file.write(f"Electrophysiological properties - parameters -----\n")
    file.write(f"   twitch_force_range_small_MU: {twitch_force_range_small_MU}\n")
    file.write(f"   twitch_force_range_large_MU: {twitch_force_range_large_MU}\n")
    file.write(f"   time_to_peak_twitch_force_range_small_MU: {time_to_peak_twitch_force_range_small_MU}\n")
    file.write(f"   time_to_peak_twitch_force_range_large_MU: {time_to_peak_twitch_force_range_large_MU}\n")



In [ ]:
# Define lerp (linear interpolation) function:
def lerp(a, b, t):
    return a + t * (b - a)

####### Generate motoneurons and their properties
motoneuron_normalized_soma_diameters = linspace(min_normalized_boundary,max_normalized_boundary,nb_motoneurons_full_pool)/(max_normalized_boundary-min_normalized_boundary) # = uniform distribution

# size_distribution_constant = 1
# size_distribution_exponent = 1/2
# size_distribution_offset = 0
# for mni in range(nb_motoneurons_full_pool):
#     motoneuron_normalized_soma_diameters[mni] = (size_distribution_constant * (lerp(0,1,motoneuron_normalized_soma_diameters[mni])**size_distribution_exponent)) + size_distribution_offset
#     motoneuron_normalized_soma_diameters[mni] = max(0, min(motoneuron_normalized_soma_diameters[mni], 1)) # clamp between 0 and 1
#     motoneuron_normalized_soma_diameters[mni] = abs(motoneuron_normalized_soma_diameters[mni] - 1)
# motoneuron_normalized_soma_diameters = motoneuron_normalized_soma_diameters[::-1] # flipping vector so that idx 0 is the smallest MN

for mni in range(nb_motoneurons_full_pool):
    motoneuron_normalized_soma_diameters[mni] = 0.581976 * (np.exp(motoneuron_normalized_soma_diameters[mni]**size_distribution_exponent)-1)
# The first number ensures that the result starts at 0

motoneuron_soma_diameters = np.zeros(nb_motoneurons_full_pool)
for mni in range(nb_motoneurons_full_pool):
    motoneuron_soma_diameters[mni] = lerp(min_soma_diameter, max_soma_diameter, motoneuron_normalized_soma_diameters[mni])

# Plot histogram of motoneuron sizes
plt.figure()
# Create the histogram
counts, bins, patches = plt.hist(motoneuron_soma_diameters, density=True)
# Multiply the counts by 100 to convert to percentage
counts_percentage = counts * 100
# Plot the histogram again with the adjusted counts
plt.clf()  # Clear the current plot
plt.hist(motoneuron_soma_diameters, density=False, weights=np.ones_like(motoneuron_soma_diameters) * (100 / len(motoneuron_soma_diameters)),
         edgecolor='white', alpha=0.75)
plt.vlines(min_soma_diameter,plt.ylim()[0],plt.ylim()[1],color='C1', label='Min soma diameter', linewidth=2)
plt.vlines(max_soma_diameter,plt.ylim()[0],plt.ylim()[1],color='C3', label='Max soma diameter', linewidth=2)
plt.legend()
plt.xlabel("Motoneuron size (soma diameter in micrometer)")
plt.ylabel("Proportion (% of total nb of motoneurons)")
plt.title("Distribution of motor neuron sizes")
new_filename = f'MN_sizes_distribution.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


plt.figure()
# Create the histogram
counts, bins, patches = plt.hist(motoneuron_normalized_soma_diameters, density=True)
# Multiply the counts by 100 to convert to percentage
counts_percentage = counts * 100
# Plot the histogram again with the adjusted counts
plt.clf()  # Clear the current plot
plt.hist(motoneuron_normalized_soma_diameters, density=False, weights=np.ones_like(motoneuron_normalized_soma_diameters) * (100 / len(motoneuron_normalized_soma_diameters)),
         edgecolor='white', alpha=0.75, color='C2')
plt.vlines(0,plt.ylim()[0],plt.ylim()[1],color='C1', label='Min soma diameter', linewidth=2)
plt.vlines(1,plt.ylim()[0],plt.ylim()[1],color='C3', label='Max soma diameter', linewidth=2)
plt.legend()
plt.xlabel("Normalized motoneuron size")
plt.ylabel("Proportion (% of total nb of motoneurons)")

plt.figure()
plt.plot(motoneuron_normalized_soma_diameters)
xlabel("Motoneuron idx")
ylabel("Nomalized MN size")
plt.ylim(-0.1,1.1)
plt.title("Size of MNs according to index")
new_filename = f'MN_sizes_according_to_idx.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# Time constant (tau)
tau_motoneurons = np.zeros(nb_motoneurons_full_pool)
refractory_period_MN = np.zeros(nb_motoneurons_full_pool)
for mni in range(nb_motoneurons_full_pool):
    tau_motoneurons[mni] = tau_constant*(motoneuron_soma_diameters[mni]**((-1)*tau_exponent))
    refractory_period_MN[mni] = lerp(refractory_period_smallest_MN,refractory_period_largest_MN, motoneuron_normalized_soma_diameters[mni])
refractory_period_MN = refractory_period_MN * second
plt.figure(figsize=(10,5))
plt.plot(tau_motoneurons, color='C0', label='Time constant')
plt.plot(refractory_period_MN*1000, color='C2', label='Refractory period')
plt.title("Time constant (tau) and refractory period distribution of simulated MNs")
plt.legend()
plt.xlabel("Motoneuron index (smallest MN is 0 ; largest MN is "+str(nb_motoneurons_full_pool-1)+")")
plt.vlines(target_force_level*nb_motoneurons_full_pool/100,plt.ylim()[0],plt.ylim()[1],color='black',linestyles='dashed',label=f'0-{target_force_level}% motoneurons')
plt.ylabel("Time (ms)")
plt.legend()
plt.show()

# Input resistance and input weight (normalized resistance)
resistance_motoneurons = np.zeros(nb_motoneurons_full_pool)
input_weight_motoneurons = np.zeros(nb_motoneurons_full_pool)
for mni in range(nb_motoneurons_full_pool):
    resistance_motoneurons[mni] = resistance_constant*(motoneuron_soma_diameters[mni]**((-1)*resistance_exponent))
    input_weight_motoneurons[mni] = resistance_motoneurons[mni]/resistance_motoneurons[0] 

fig, ax1 = plt.subplots(figsize=(10,5))
ax1.plot(resistance_motoneurons, color='C1', label = 'Input resistance')
ax1.set_ylabel("Resistance (ohms)", color='C1')
ax2 = ax1.twinx()
ax2.plot(input_weight_motoneurons, color='C3', label = 'Input weight')
ax2.set_ylabel("Input weight", color='C3')
ax2.set_ylim([0,1])
plt.vlines(target_force_level*nb_motoneurons_full_pool/100,plt.ylim()[0],plt.ylim()[1],color='black',linestyles='dashed',label=f'0-{target_force_level}% motoneurons')
ax1.legend(loc='upper right', bbox_to_anchor=(1, 1))
ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.9))
ax1.set_xlabel("Motoneuron index (smallest MN is 0 ; largest MN is "+str(nb_motoneurons_full_pool-1)+")")
plt.title("Resistance distribution of MNs")
plt.show()

# Twitch force and duration and electromechanical delay (implemented as zeros before the kernel)
twitch_force_motoneurons = np.zeros(nb_motoneurons_full_pool)
electromechanical_delay_motoneurons = np.zeros(nb_motoneurons_full_pool)
twitch_duration_motoneurons = np.zeros(nb_motoneurons_full_pool)
twitch_convolution_window = [None] * nb_motoneurons_full_pool
for mni in range(nb_motoneurons_full_pool):
    twitch_force_motoneurons[mni] = lerp(twitch_force_range_small_MU,twitch_force_range_large_MU,motoneuron_normalized_soma_diameters[mni])
    electromechanical_delay_motoneurons[mni] = 1/(axonal_conduction_velocity_constant*(motoneuron_soma_diameters[mni]**(axonal_conduction_velocity_exponent)))
    twitch_duration_motoneurons[mni] = lerp(time_to_peak_twitch_force_range_small_MU,time_to_peak_twitch_force_range_large_MU,motoneuron_normalized_soma_diameters[mni])
    # twitch_convolution_window[mni] = ((fsamp * twitch_duration_motoneurons[mni] * (1/2))**-1) * windows.hann(round(fsamp * twitch_duration_motoneurons[mni])) *  twitch_force_motoneurons[mni]
    twitch_convolution_window[mni] = windows.hann(round(fsamp * twitch_duration_motoneurons[mni])) * twitch_force_motoneurons[mni]
    # Extend the ramp down phase of the twitch so that it is five times the size of the ramp up phase (crude approximation based on Figure 1 of Raikova 2023. Full model explained in the paper https://www.sciencedirect.com/science/article/pii/S1050641123000330?via%3Dihub)
    twitch_force_down_temp = twitch_convolution_window[mni][int(np.round(len(twitch_convolution_window[mni])/2)):len(twitch_convolution_window[mni])]
    twitch_convolution_window[mni] = twitch_convolution_window[mni][:int(np.round(len(twitch_convolution_window[mni])/2))] # remove the down part (to be added in a few lines later)
    original_indices = np.linspace(0, len(twitch_force_down_temp) - 1, num=len(twitch_force_down_temp)) # Create the indices of the original and new vectors
    new_indices = np.linspace(0, len(twitch_force_down_temp) - 1, num = multiplication_of_twitch_force_down_time * len(twitch_force_down_temp) ) # Desired length of the new vector
    twitch_force_down_stretched = np.interp(new_indices, original_indices, twitch_force_down_temp) # Perform linear interpolation
    twitch_convolution_window[mni] = np.append(twitch_convolution_window[mni],twitch_force_down_stretched)
    # insert zeros corresponding to electromechanical delay
    delay_insamples = int(np.round(electromechanical_delay_motoneurons[mni]*fsamp))
    twitch_convolution_window[mni] = np.append(np.zeros(delay_insamples), twitch_convolution_window[mni])
    # Double the length of the vector with only zeros at the beginning, so that the convolution causes the force twitch to happen after each spike
    twitch_convolution_window[mni] = np.append(np.zeros(len(twitch_convolution_window[mni])),twitch_convolution_window[mni])
# figure of twitch force, twitch duration, electromechanical delay
fig, ax1 = plt.subplots(figsize=(10,5))
ax1.plot(twitch_force_motoneurons, color='C1', label = 'Twitch force')
ax1.set_ylabel("Twitch torque (milliNewton/meter)", color='C1')
ax2 = ax1.twinx()
ax2.plot(twitch_duration_motoneurons*1e3, color='C2', label = 'Twitch duration')
ax2.plot(electromechanical_delay_motoneurons*1e3, color='C4', label = 'Electromechanical delay')
ax2.set_ylabel("Time (ms)")
plt.vlines(target_force_level*nb_motoneurons_full_pool/100,plt.ylim()[0],plt.ylim()[1],color='black',linestyles='dashed',label=f'0-{target_force_level}% motoneurons')
ax1.legend(loc='upper right', bbox_to_anchor=(1, 1))
ax2.legend(loc='upper right', bbox_to_anchor=(1, 0.9))
ax1.set_xlabel("Motoneuron index (smallest MN is 0 ; largest MN is "+str(nb_motoneurons_full_pool-1)+")")
plt.title("MNs force twitch properties")
new_filename = f'Twitch_force_properties.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')
# convolution windows
fig, ax = plt.subplots(figsize=(10,5))
for mni in range(nb_motoneurons_full_pool):
    if mni == 0:
        plt.plot(twitch_convolution_window[mni], color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 1, linewidth = 1.5, label = "smallest simulated motor unit")
    elif mni == nb_motoneurons_full_pool-1:
        plt.plot(twitch_convolution_window[mni], color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 1, linewidth = 1.5, label = "largest simulated motor unit")
    elif mni == int(np.ceil(nb_motoneurons_full_pool*target_force_level/100)):
        plt.plot(twitch_convolution_window[mni], color='black', alpha = 1, linewidth = 2.5, label=f'{target_force_level}% motoneurons')
    else:
        plt.plot(twitch_convolution_window[mni], color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5, linewidth = 2)
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Torque (milliNewton/meter)")
plt.legend()
plt.title("Kernel for twitch torque convolution")
new_filename = f'Twitch_force_convolution_kernels.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Sanity check of the convolution process to obtain force
binary_spike_test = np.zeros(1500)
binary_spike_test[[100,200,700]] = 1
conv_kernel_test = np.copy(twitch_convolution_window[0])
conv_kernel_test = conv_kernel_test / max(conv_kernel_test)
plt.figure()
plt.plot(binary_spike_test,label="spike train")
test_conv = np.convolve(binary_spike_test, conv_kernel_test, mode='same')
plt.plot(test_conv,label="force produced")
plt.xlabel("Time (ms)")
plt.title("Testing the convolution of the spike train with the twitch force kernel (visual sanity check)")
plt.legend()


In [ ]:
# Define a low-pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
# When rounding 2.7 for example, 70% to get 3 and 30% chance to get 2
def probabilistic_round(number):
    lower = int(number)  # The lower integer
    upper = lower + 1    # The upper integer
    decimal_part = number - lower
    
    return upper if random.random() < decimal_part else lower

In [ ]:
# Initialize values

# # Initialize network - groups of motoneurons and synapses
# eqs_motoneuron = '''
# dv/dt = ((I(t,i)*input_weight)-v)/tau: 1 (unless refractory)
# tau : second
# refractory_period : second
# input_weight : 1
# '''

# same input for all motoneurons for initial optimization
eqs_motoneuron = '''
dv/dt = ((I(t)*input_weight)-v)/tau: 1 (unless refractory)
tau : second
refractory_period : second
input_weight : 1
'''

# Groups of neurons
motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                          threshold='v>voltage_thresh',
                          reset='v=voltage_rest',
                          refractory='refractory_period',
                          method='exact')
motoneurons.tau = tau_motoneurons*(1/1000)*second # convert to ms
motoneurons.refractory_period = refractory_period_MN
motoneurons.input_weight = input_weight_motoneurons

# Display MN properties
plt.plot(motoneurons.tau)
plt.plot(motoneurons.input_weight)
plt.plot(motoneurons.refractory_period)
plt.xlabel("MN indx")
plt.title("MNs properties")
plt.vlines(target_force_level*nb_motoneurons_full_pool/100,plt.ylim()[0],plt.ylim()[1],color='black',linestyles='dashed',label=f'0-{target_force_level}% motoneurons')
plt.legend(["Time constant (seconds)","Input weight (0-1 scaling factor)","refractory period (seconds)",f'0-{target_force_level}% motoneurons'])
new_filename = f'Properties_MN.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Monitors
monitor_state_motoneurons = StateMonitor(motoneurons, variables=True, record=True)
monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)

In [ ]:
# GET SPIKE TRAINS AND BINARY SPIKE TRAINS
def Get_binary_spike_trains(spike_monitor, sim_duration):
    # Define time bins
    time_bins = np.arange(0, int(np.round((sim_duration*fsamp)))*second) * ms

    # Retrieve spikes and get binary spike trains
    spike_trains = []
    for mni in range(nb_motoneurons_full_pool):
        spike_trains.append(spike_monitor.spike_trains()[mni])
    
    # Initialize the binary spike train array
    binary_spike_trains = {}
    binary_spike_trains = np.zeros((nb_motoneurons_full_pool, len(time_bins)))
    # Convert spike times to binary spike train
    for neuron_idx in range(nb_motoneurons_full_pool):
        spikes = spike_trains[neuron_idx]
        spike_indices = np.searchsorted(time_bins, spikes)
        binary_spike_trains[neuron_idx, spike_indices-1] = 1 #-1 because of offset due to 0-indexing
    
    return spike_trains, binary_spike_trains

In [ ]:
# CONVOLVE TO GET FORCE
def Convolve_to_get_force(binary_spike_trains, corresponding_MN_idx, absolute_or_normalized):

    force_per_MU = []
    force_total = np.zeros(binary_spike_trains.shape[1])

    if ndim(binary_spike_trains) > 1:
        if shape(binary_spike_trains)[0] != len(corresponding_MN_idx):
            print("Error = the number of MU indices provided do not match with the number of rows in the binary matrix")
        for mni in range(shape(binary_spike_trains)[0]):
            # 'same' mode means the output length will be the same as the input length
            if absolute_or_normalized == 'absolute':
                temp_force = np.convolve(binary_spike_trains[mni,:], twitch_convolution_window[corresponding_MN_idx[mni]], mode='same')
            elif absolute_or_normalized == 'normalized':
                temp_force = np.convolve(binary_spike_trains[mni,:], twitch_convolution_window_normalized[corresponding_MN_idx[mni]], mode='same')
            force_per_MU.append(temp_force)
            # ax.plot(force_total, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5)
            force_total = force_total + temp_force
    else:
        # if len(corresponding_MN_idx) != 1:        # len() doesn't work on a unique element
        #     print("Error = the number of MU indices provided do not match with the number of rows in the binary matrix")
        if absolute_or_normalized == 'absolute':
            temp_force = np.convolve(binary_spike_trains, twitch_convolution_window[corresponding_MN_idx[mni]], mode='same')
        elif absolute_or_normalized == 'normalized':
            temp_force = np.convolve(binary_spike_trains, twitch_convolution_window_normalized[corresponding_MN_idx[mni]], mode='same')
        force_per_MU.append(temp_force)
        force_total = force_total + temp_force
    return force_total

In [ ]:
# Get max tetanic force of the simulated pool => by sending a very very high input to all MNs and recording the resulting force
# The force comes from the estimated peak torque values of individual motor units in the human tibialis anterior, so the low MVC values are not so unrealistic
# Reset simulation
start_scope() # Re-initialize the simulation

# Set extremely (unrealistically high) input
MVC_duration = 5 # in second
synaptic_input_for_max_force = np.ones(MVC_duration*fsamp) * 10 # 10 is pretty good to get a mean firing rates of ~40pps (depending on the distribution of MN sizes) for the estimated MVC, which is physiologicaly plausible (https://pubmed.ncbi.nlm.nih.gov/6887053/)
I = TimedArray(synaptic_input_for_max_force, dt=1*ms)
# To estimate 100% MVC, if for example motor units are simulated up to 20% of largest motor units, resulting force will be multiplied by 1/0.2 = 5)
nb_motoneurons_full_pool_to_simulate_for_MVC = nb_motoneurons_full_pool * (1/(max_normalized_boundary/100))

eqs_motoneuron = '''
dv/dt = ((I(t)*input_weight)-v)/tau: 1 (unless refractory)
tau : second
refractory_period : second
input_weight : 1
'''
# Groups of neurons
motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                        threshold='v>voltage_thresh',
                        reset='v=voltage_rest',
                        refractory='refractory_period',
                        method='exact')
motoneurons.tau = tau_motoneurons*(1/1000)*second # convert to ms
motoneurons.refractory_period = refractory_period_MN
motoneurons.input_weight = input_weight_motoneurons
# Initialize voltage of MNs and Renshaw cells randomly
motoneurons.v = rand() # uniform distribution between 0 and 1
monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)
# Run simulation
run(MVC_duration * second)
MVC_sim_samples = MVC_duration * fsamp
# Get spike trains
spike_trains, binary_spike_trains = Get_binary_spike_trains(monitor_spikes_motoneurons, MVC_duration)
# Get force
MVC_force = Convolve_to_get_force(binary_spike_trains,np.arange(nb_motoneurons_full_pool),'absolute')
# Force per MU
force_total = zeros(len(binary_spike_trains[0,:]))
# Remove edges to get rid of artifacts (from convolution and/or synchronization of motor units)
MVC_force = MVC_force[:MVC_sim_samples-fsamp]
MVC_force = MVC_force[fsamp:]
force_total = force_total[:MVC_sim_samples-fsamp]
force_total = force_total[fsamp:]

fig, ax = plt.subplots(figsize=(10, 5))
# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')
for mni in range(nb_motoneurons_full_pool):
    # 'same' mode means the output length will be the same as the input length
    temp_force = np.convolve(binary_spike_trains[mni,:], twitch_convolution_window[mni], mode='same')/1000
    temp_force = temp_force[:MVC_sim_samples-fsamp]
    temp_force = temp_force[fsamp:]
    force_total += temp_force
    ax.plot(force_total, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5)
ax.plot(force_total, color='black', alpha = 1, linewidth = 2)
plt.title(f"Reconstructed torque (convolving spike train with twitch torque kernel)")
plt.ylabel("Torque (Newton/meter)")
plt.xlabel("Time (ms)")
new_filename = f'Max_force_for_simulated_MNs.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

### Get discharge characteristics of motoneurons
mean_firing_rate = {}
std_firing_rate = {}
fig, axs = plt.subplots()
firing_rates = []
highest_ISIs = []
for mni in range(nb_motoneurons_full_pool):
    if len(spike_trains[mni]) <= 1:
        highest_ISIs.append(MVC_duration*fsamp)
    else:
        highest_ISIs.append(max(diff(spike_trains[mni])))
    firing_rate_temp = len(spike_trains[mni]) / MVC_duration
    firing_rates.append(firing_rate_temp)
# Convert to a numpy array for easier calculations
firing_rates = np.array(firing_rates)
# Calculate mean and standard deviation of the firing rates
mean_firing_rate = np.mean(firing_rates)
std_firing_rate = np.std(firing_rates)
# Motoneurons' firing rates results
axs.hist(firing_rates, edgecolor='white', alpha=0.75)
axs.axvline(x = mean_firing_rate, linestyle='--', linewidth=2, label='Mean firing rate')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate - Very high excitatory input simulation (for MVC estimation)")
 
MVC_force = MVC_force/1000
max_MVC_force = np.max(MVC_force)
print(f'Estimated MVC (Newton/meter) = {max_MVC_force}')
target_force_level_absolute_value = max_MVC_force * target_force_level/100
print(f'Target torque level (Netwon/meter) = {target_force_level_absolute_value}')

In [ ]:
# Convert the twitch torque values of MUs in % of MVC
total_peak_forces = np.sum((twitch_force_motoneurons/1000)/max_MVC_force)
correcting_peak_force_value = 1 / total_peak_forces

twitch_force_motoneurons_normalized = twitch_force_motoneurons/1000
twitch_convolution_window_normalized = {}

for mni in range(nb_motoneurons_full_pool):
    twitch_force_motoneurons_normalized[mni] = (twitch_force_motoneurons_normalized[mni]/max_MVC_force)*correcting_peak_force_value
    twitch_convolution_window_normalized[mni] = ((twitch_convolution_window[mni]/1000)/max_MVC_force)*correcting_peak_force_value*100 # *100 to express in % MVC

# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')
# convolution windows
fig, ax = plt.subplots(figsize=(10,5))
for mni in range(nb_motoneurons_full_pool):
    if mni == 0:
        plt.plot(twitch_convolution_window_normalized[mni], color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 1, linewidth = 1, label = "smallest simulated motor unit")
    elif mni == nb_motoneurons_full_pool-1:
        plt.plot(twitch_convolution_window_normalized[mni], color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 1, linewidth = 1, label = "largest simulated motor unit")
    elif mni == int(np.ceil(nb_motoneurons_full_pool*target_force_level/100)):
        plt.plot(twitch_convolution_window_normalized[mni], color='black', alpha = 1, linewidth = 2.5, label=f'{target_force_level}% motoneurons')
    else:
        plt.plot(twitch_convolution_window_normalized[mni], color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5, linewidth = 2)
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Torque (% of MVC)")
plt.legend()
plt.title("Kernel for NORMALIZED twitch torque convolution")
new_filename = f'Twitch_force_convolution_kernels_normalized.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
# Define force target
if (target_type == 'plateau'):
    print("Plateau force target")
    target_force = ones(int(duration_with_ignored_window * fsamp))*target_force_level
elif (target_type == 'sinusoid'):
    print("Sinusoidal force target")
    target_force = ones(int(duration_with_ignored_window * fsamp))*target_force_level
    sinusoids_temp =  np.linspace(0, duration_with_ignored_window, int(duration_with_ignored_window*fsamp), endpoint=False)
    sinusoids_temp = sinusoids_temp / second
    sinusoids_temp = (target_force * 0.25) * np.sin(2 * np.pi * target_force_sin_freq * sinusoids_temp)
    target_force = target_force + sinusoids_temp
elif (target_type == 'trapezoid'):
    print("Trapezoidal force target")
    target_force = np.zeros(int(window_beginning_ignore*fsamp))
    ramp_up = linspace(0,target_force_level,ramp_duration*fsamp)
    target_force = np.append(target_force,ramp_up)
    plateau = ones(int(plateau_duration * fsamp))*target_force_level
    target_force = np.append(target_force,plateau)
    ramp_down = linspace(target_force_level,0,ramp_duration*fsamp)
    target_force = np.append(target_force,ramp_down)
    target_force = np.append(target_force,np.zeros(int(window_end_ignore*fsamp)))
else:
    print("Please select a valid target type type")
    sys.exit()

plt.plot(target_force, color='black', alpha = 0.5, linewidth = 2)
plt.xlabel("Time (ms)")
plt.ylabel("Target force (% MVC)")
plt.title("Target force")
new_filename = f'Target_force.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
time = linspace(0,duration_with_ignored_window/second,int(duration_with_ignored_window/second*fsamp))

In [ ]:
# INHIBITORY INPUT
Wind_s = 1/low_pass_filter_of_inhibitory_input  # hanning window duration.
HanningW = 2 / round(fsamp * Wind_s) * windows.hann(round(fsamp * Wind_s))  # unitary area

# Create inhibitory input
inhib_input = {}
if inhibitory_input_source == 'generate_synthetic_input':
    for inhibiti in range(nb_inhibitory_input):
        inhib_input[inhibiti] = []
        temp_inhib = np.random.normal(0, 1, int(duration_with_ignored_window * fsamp))
        temp_inhib[int(duration_with_ignored_window * fsamp)-int(np.round(window_beginning_ignore/1)):int(duration_with_ignored_window * fsamp)] = 0
        temp_inhib[0:int(np.round(window_beginning_ignore/1))] = 0
        temp_inhib = lowpass_filter(temp_inhib, low_pass_filter_of_inhibitory_input, fsamp)
        # temp_inhib = filtfilt(HanningW, 1, temp_inhib * fsamp)
        temp_inhib = temp_inhib - np.mean(temp_inhib)
        temp_inhib = temp_inhib / np.std(temp_inhib)
        temp_inhib = temp_inhib * inhibitory_input_std
        temp_inhib = temp_inhib + inhibitory_input_mean
        if prevent_inhibition_from_being_positive:
            temp_inhib[temp_inhib > 0] = 0
        inhib_input[inhibiti].append(temp_inhib)
elif inhibitory_input_source == 'load_synthetic_input':
    synthetic_signals_dataframe = pd.read_csv(inhibitory_input_sourcefile)
    if inhibitory_input_sourcefile_fsamp != fsamp:
        from scipy.interpolate import interp1d
        # Calculate the time array for the original signal
        loaded_signal_time = np.arange(len(synthetic_signals_dataframe)) / inhibitory_input_sourcefile_fsamp
        # Calculate the number of samples in the resampled signal
        number_of_samples = int(len(synthetic_signals_dataframe) * fsamp / inhibitory_input_sourcefile_fsamp)
        # Calculate the time array for the resampled signal
        resampled_time = np.linspace(loaded_signal_time[0], loaded_signal_time[-1], number_of_samples)
    for inhibiti in range(nb_inhibitory_input):
        inhib_input[inhibiti] = []
        temp_inhib = synthetic_signals_dataframe[f'{inhibiti}'].values
        if inhibitory_input_sourcefile_fsamp != fsamp:
            # Create an interpolation function
            resample_loaded_signal_function = interp1d(loaded_signal_time, temp_inhib, kind='linear')
            temp_inhib = resample_loaded_signal_function(resampled_time)
        temp_inhib = temp_inhib[:len(time)] # cut the signal for it to be the right size
        temp_inhib = temp_inhib * inhibitory_input_std
        temp_inhib = temp_inhib + inhibitory_input_mean
        if prevent_inhibition_from_being_positive:
            temp_inhib[temp_inhib > 0] = 0
        inhib_input[inhibiti].append(temp_inhib)

plt.figure()
for inhibiti in range(nb_inhibitory_input):
    plt.plot(np.transpose(inhib_input[inhibiti]), alpha = 0.5, label=f'Inhibitory input #{inhibiti}', color = 'C4')
plt.xlabel("Time (ms)")
plt.ylabel("Inhibitory input(s)")
plt.legend()
new_filename = f'Inhibitory_input_signal.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

if (inhibition_distribution == 'homogeneous'):
    proportion_of_MN_affected_by_each_inhibitory_input = 100
elif (inhibition_distribution == 'heterogeneous'):
    proportion_of_MN_affected_by_each_inhibitory_input = proportion_of_MN_affected_by_each_inhibitory_input_original
else:
    print("Please select a valid inhibition distribution type ('homogeneous' or 'heterogeneous')")
    sys.exit()

MN_inhibition_weights_curve = np.zeros(nb_motoneurons_full_pool)
if (inhibition_weight_distribution == 'uniform'):
    inhibition_exponent_weights = 0
    inhibition_offset_weights = 0
    inhibition_constant_weights = 1
elif (inhibition_weight_distribution == 'exponential'):
    inhibition_exponent_weights = inhibition_exponent_weights_original
    inhibition_offset_weights = inhibition_offset_weights_original
    inhibition_constant_weights = inhibition_constant_weights_original
else:
    print("Please select a valid inhibition weights distribution type ('uniform' or 'exponential')")
    sys.exit()
for mni in range(nb_motoneurons_full_pool):
    MN_inhibition_weights_curve[mni] = inhibition_constant_weights * (1-motoneuron_normalized_soma_diameters[mni])**inhibition_exponent_weights + inhibition_offset_weights
    MN_inhibition_weights_curve[mni] = max(0,min(MN_inhibition_weights_curve[mni],1))

plt.plot(MN_inhibition_weights_curve, color='C4')
ylim([-0.1,1.1])
ylabel("Inhibition weights")
xlabel("MN index")


# Distribute inhiitory input
MN_receiving_inhibitory_input = {}
for inhibiti in range(nb_inhibitory_input):
    MN_receiving_inhibitory_input[inhibiti] = []
    MN_receiving_inhibitory_input[inhibiti].append(np.random.choice(range(nb_motoneurons_full_pool),
                                                                      int(np.round(nb_motoneurons_full_pool*(proportion_of_MN_affected_by_each_inhibitory_input/100))),
                                                                      replace=False))
    MN_receiving_inhibitory_input[inhibiti][0] = np.sort(MN_receiving_inhibitory_input[inhibiti][0])
MN_inhibition_weights = {}
for inhibiti in range(nb_inhibitory_input):
    MN_inhibition_weights[inhibiti] = np.zeros(nb_motoneurons_full_pool)
    for mni in MN_receiving_inhibitory_input[inhibiti][0]:
        MN_inhibition_weights[inhibiti][mni] = MN_inhibition_weights_curve[mni]

fig, ax = plt.subplots(1,1, figsize=(15,4))
x_plot_mns = range(nb_motoneurons_full_pool)
bottom_barplot = np.zeros(nb_motoneurons_full_pool)
for inhibiti in range(nb_inhibitory_input):
    vstack_inhibi_temp = MN_inhibition_weights[inhibiti]
    ax.bar(x_plot_mns,
        vstack_inhibi_temp,
        bottom = bottom_barplot,
        label = f"inhibitory input #{inhibiti}", color="C4")
    bottom_barplot += vstack_inhibi_temp
ax.set_xlabel('Motoneurons')
ax.set_ylabel('Inhibitory input weights')
plt.suptitle("Distribution of inhibitory inputs according to INDICES")
plt.legend()
new_filename = f'Inhibitory_input_distrib_relative_to_indices.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure()
plt.scatter(motoneuron_normalized_soma_diameters,MN_inhibition_weights_curve, color='C4')
ylim([-0.1,1.1])
xlabel("Normalized MN size")
ylabel('Inhibitory input weights')
plt.title("Distribution of inhibitory inputs according to SIZE")
new_filename = f'Inhibitory_input_distrib_relative_to_size.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()
    

In [ ]:
inhibitory_input_power_integral_0_5_hz = []
for inhibiti in range(nb_inhibitory_input):
    inhib_input_temp = np.copy(np.squeeze(np.array(inhib_input[inhibiti])))
    # remove artifacts of low pass filter
    inhib_input_temp[0:int(window_beginning_ignore*fsamp)] = inhibitory_input_mean
    inhib_input_temp[len(time)-int(window_end_ignore*fsamp):len(time)] = inhibitory_input_mean
    # Normalize
    inhib_input_temp = ((inhib_input_temp - np.mean(inhib_input_temp)) / np.std(inhib_input_temp)) * inhibitory_input_std
    N = len(inhib_input_temp)
    yf = fft(inhib_input_temp)
    xf = fftfreq(N, 1 / fsamp)
    power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
    inhibitory_input_power_integral = np.sum(power_spectrum_temp)
    if inhibiti == 0:
        idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
    power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
    inhibitory_input_power_integral_0_5_hz.append(np.sum(power_spectrum_temp_0_5hz))
    plt.plot(xf[:N//2], power_spectrum_temp, color = 'C4', alpha = 1/nb_inhibitory_input)
inhibitory_input_power_integral_0_5_hz_mean = np.mean(inhibitory_input_power_integral_0_5_hz)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Power spectrum of the inhibitory inputs")
plt.xlim([0,20])
new_filename = f'Signal_inhibitory_input_power_spectrum.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
# Generate independent input to MNs
plt.figure()
independent_input_noise = randn(nb_motoneurons_full_pool,len(time)) # noise input, with mean zero and std 1 (default setting)
# Plot the frequency domain independent noise signal
# Apply low-pass filter to the Gaussian noise
independent_input_noise_power_integral_0_5_hz = []
for mni in range(nb_motoneurons_full_pool):
    independent_input_noise[mni,:] = lowpass_filter(independent_input_noise[mni,:], low_pass_filter_of_independent_noise, fsamp,3)
    # remove artifacts of low pass filter
    independent_input_noise[mni,:][0:window_beginning_ignore*fsamp] = 0
    independent_input_noise[mni,:][len(time)-(window_end_ignore*fsamp):len(time)] = 0
    independent_input_noise[mni,:] = ((independent_input_noise[mni,:] - np.mean(independent_input_noise[mni,:])) / np.std(independent_input_noise[mni,:])) * independent_noise_amplitude
    N = len(independent_input_noise[mni,:])
    yf = fft(independent_input_noise[mni,:])
    xf = fftfreq(N, 1 / fsamp)
    power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
    independent_noise_power_integral = np.sum(power_spectrum_temp)
    if mni == 0:
        idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
    power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
    independent_input_noise_power_integral_0_5_hz.append(np.sum(power_spectrum_temp_0_5hz))
    plt.plot(xf[:N//2], power_spectrum_temp, color = 'C0', alpha = 1/nb_motoneurons_full_pool)
independent_input_noise_power_integral_0_5_hz_mean = np.mean(independent_input_noise_power_integral_0_5_hz)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Power spectrum of the independent noise inputs")
plt.xlim([0,80])
new_filename = f'Signal_independent_noise_power_spectrum.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
# Initialize input current + # Store initial control signal
excit_I = ((target_force / target_force_level) * common_input_baseline)**(1/3) # **(1/2) or **(1/3) to make the relationship non linear and helping the optimizer converge when force is changing
initial_excit_signal = excit_I.copy()
samples_of_interest = list(range((window_beginning_ignore*fsamp),len(time)-(window_end_ignore*fsamp)))

# Define the cost function
def cost_function(output_force_var, target_force_var):
    return np.sum(abs(output_force_var - target_force_var))

# Gradient descent parameters
learning_rate = 0.2
# num_optimization_iterations = 10

# Adam optimizer parameters
initial_alpha = learning_rate  # Initial learning rate
beta1 = 0.1 #0.9
beta2 = 0.5 #0.999
epsilon = 1e-8
m = np.zeros_like(excit_I)
v = np.zeros_like(excit_I)
timestep = 0

# Lists to store error and output force for plotting
errors = []
initial_output_force = None

# Reset simulation
start_scope() # Re-initialize the simulation

optimized_excit_signal = np.copy(initial_excit_signal)
best_cost = np.Inf
best_iter_idx = 0
# Optimization loop using Adam optimizer
for iteration in range(num_optimization_iterations):
    timestep += 1

    # Low-pass filter the current excitatory input signal to prevent the "learning/optimization" towards an oscillating common input
    # 1 hz cut-off
    excit_I = lowpass_filter(excit_I, 1, fsamp)
    # remove artifacts of low pass filter
    excit_I[0:int(np.round(window_beginning_ignore*fsamp)*0.5)] = 0
    excit_I[len(time)-int(np.round((window_end_ignore*fsamp)*0.5)):len(time)] = 0
    # Prevent common input from going negative (it happens at the beginning of slopes)
    excit_I[excit_I < 0] = 0

    # Update input current in the neuron group
    net_input_I = np.zeros((nb_motoneurons_full_pool,len(time)))
    for mni in range(nb_motoneurons_full_pool):
        net_input_I[mni,:] = excit_I
        net_input_I[mni,:] += independent_input_noise[mni,:]
        if (inhib_before_force_optimization == True):
            for inhibiti in range(nb_inhibitory_input):
                net_input_I[mni,:] += inhib_input[inhibiti][0]*MN_inhibition_weights[inhibiti][mni]
    net_input_I = np.transpose(net_input_I)
    I = TimedArray(net_input_I,dt=1*ms)

    # same input for all motoneurons for initial optimization
    eqs_motoneuron = '''
    dv/dt = ((I(t,i)*input_weight)-v)/tau: 1 (unless refractory)
    tau : second
    refractory_period : second
    input_weight : 1
    '''
    # Groups of neurons
    motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                            threshold='v>voltage_thresh',
                            reset='v=voltage_rest',
                            refractory='refractory_period',
                            method='exact')
    motoneurons.tau = tau_motoneurons*(1/1000)*second # convert to ms
    motoneurons.refractory_period = refractory_period_MN
    motoneurons.input_weight = input_weight_motoneurons
    # Initialize voltage of MNs and Renshaw cells randomly
    motoneurons.v = rand() # uniform distribution between 0 and 1

    monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)

    # Run simulation
    run(duration_with_ignored_window)

    # Get spike trains
    spike_trains, binary_spike_trains = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)

    # Get force
    output_force = Convolve_to_get_force(binary_spike_trains,np.arange(nb_motoneurons_full_pool),'normalized')
    # if low_pass_filter_force:
        # output_force = lowpass_filter(output_force, low_pass_filter_of_force_cutoff, fsamp)
    if iteration == 0:
        initial_output_force = output_force
    # During the learning phase, low-pass filtering the force helps the algorithm converge towards a better common input "solution"
    output_force = lowpass_filter(output_force, 1, fsamp)

    # Calculate cost
    output_force_windowed = output_force[samples_of_interest]
    target_force_windowed = target_force[samples_of_interest]
    cost = cost_function(output_force_windowed, target_force_windowed)/len(samples_of_interest)
    errors.append(cost)
    print(f'Iteration {iteration + 1}, Cost: {np.round(cost*100)/100} (mean error in % of MVC)')
    if cost < best_cost:
        print(f'New best control signal')
        best_cost = cost
        optimized_excit_signal = np.copy(excit_I)
        best_iter_idx = iteration

    # Adjust learning rate inversely proportional to the error
    alpha = min(cost*learning_rate*0.5,learning_rate)
    # alpha = learning_rate
    
    # Compute gradients
    gradients = (output_force - target_force)

    # Adam optimizer updates
    m = beta1 * m + (1 - beta1) * gradients
    v = beta2 * v + (1 - beta2) * (gradients ** 2)
    m_hat = m / (1 - beta1 ** timestep)
    v_hat = v / (1 - beta2 ** timestep)
    excit_I -= alpha * m_hat / (np.sqrt(v_hat) + epsilon)


In [ ]:
# Create common noise (fluctuation of the common input)
if common_input_source  == 'generate_synthetic_input':
    common_noise = randn(len(time)) # noise input, with mean zero and std 1 (default setting)
    common_noise = lowpass_filter(common_noise, low_pass_filter_of_common_noise, fsamp)
    # remove artifacts of low pass filter
    common_noise[0:window_beginning_ignore*fsamp] = 0
    common_noise[len(common_noise)-(window_end_ignore*fsamp):len(common_noise)] = 0
elif common_input_source == 'load_synthetic_input':
    synthetic_signals_dataframe = pd.read_csv(common_input_sourcefile)
    common_noise = synthetic_signals_dataframe[f'{synthetic_signals_dataframe.shape[1]-1}'].values
    if common_input_sourcefile_fsamp != fsamp:
        from scipy.interpolate import interp1d
        # Calculate the time array for the original signal
        loaded_signal_time = np.arange(len(synthetic_signals_dataframe)) / common_input_sourcefile_fsamp
        # Calculate the number of samples in the resampled signal
        number_of_samples = int(len(synthetic_signals_dataframe) * fsamp / common_input_sourcefile_fsamp)
        # Calculate the time array for the resampled signal
        resampled_time = np.linspace(loaded_signal_time[0], loaded_signal_time[-1], number_of_samples)
        # Create an interpolation function
        resample_loaded_signal_function = interp1d(loaded_signal_time, common_noise, kind='linear')
        common_noise = resample_loaded_signal_function(resampled_time)
    common_noise = common_noise[:len(time)] # cut the signal for it to be the right size
# Normalize
common_noise = ((common_noise - np.mean(common_noise)) / np.std(common_noise)) * common_input_fluctuations_amplitude
            


plt.figure()
plt.plot(common_noise, label="common noise", color='C3')
plt.xlabel("Time (ms)")
plt.ylabel("Common noise amplitude")
plt.legend()

plt.figure()
N = len(common_noise)
yf = fft(common_noise)
xf = fftfreq(N, 1 / fsamp)
power_spectrum_temp = (np.abs(yf[:N//2])**2) / N
common_noise_power_integral = np.sum(power_spectrum_temp)
idx_corresponding_to_5hz = int(np.round((N/fsamp)*5))
power_spectrum_temp_0_5hz = power_spectrum_temp[:idx_corresponding_to_5hz]
common_noise_power_intergal_0_5_hz = np.sum(power_spectrum_temp_0_5hz)
plt.plot(xf[:N//2], power_spectrum_temp, color = 'C3', alpha = 1)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Power spectrum of the common noise (fluctuations in common input)")
plt.xlim([0,10])
new_filename = f'Common_noise_power_spectrum.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# # Re-run the simulation with the parameter ending up with the lowest cost, with a low-pass filter of the optimized signal + common noise
# lowpass_filtered_optimized_excit_signal = lowpass_filter(optimized_excit_signal,1,fsamp) # 1hz low-pass filter
# # remove artifacts of low pass filter
# lowpass_filtered_optimized_excit_signal[0:int(np.round(window_beginning_ignore*fsamp)*0.5)] = 0
# lowpass_filtered_optimized_excit_signal[len(time)-int(np.round((window_end_ignore*fsamp)*0.5)):len(time)] = 0
# common_input = lowpass_filtered_optimized_excit_signal + common_noise

common_input = optimized_excit_signal + common_noise

net_input_I = np.zeros((nb_motoneurons_full_pool,len(time)))
for mni in range(nb_motoneurons_full_pool):
    net_input_I[mni,:] = common_input
    net_input_I[mni,:] += independent_input_noise[mni,:]
    for inhibiti in range(nb_inhibitory_input):     # Add the inhibition
        net_input_I[mni,:] += inhib_input[inhibiti][0]*MN_inhibition_weights[inhibiti][mni]
net_input_I = np.transpose(net_input_I)
I = TimedArray(net_input_I,dt=1*ms)

# same input for all motoneurons for initial optimization
eqs_motoneuron = '''
dv/dt = ((I(t,i)*input_weight)-v)/tau: 1 (unless refractory)
tau : second
refractory_period : second
input_weight : 1
'''
# Groups of neurons
motoneurons = NeuronGroup(nb_motoneurons_full_pool, eqs_motoneuron,
                        threshold='v>voltage_thresh',
                        reset='v=voltage_rest',
                        refractory='refractory_period',
                        method='exact')
motoneurons.tau = tau_motoneurons*(1/1000)*second # convert to ms
motoneurons.refractory_period = refractory_period_MN
motoneurons.input_weight = input_weight_motoneurons
# Initialize voltage of MNs and Renshaw cells randomly
motoneurons.v = rand() # uniform distribution between 0 and 1

monitor_spikes_motoneurons = SpikeMonitor(motoneurons, record=True)

# Run simulation
run(duration_with_ignored_window)

# Get spike trains
spike_trains, binary_spike_trains = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)

# Get force
output_force = Convolve_to_get_force(binary_spike_trains,np.arange(nb_motoneurons_full_pool),'normalized')
if low_pass_filter_force:
    output_force = lowpass_filter(output_force, low_pass_filter_of_force_cutoff, fsamp)

In [ ]:
### Restrict window of analyzis
samples_for_analyzis = []
if target_type == 'trapezoid':
    if analyzis_window == 'plateau':
        samples_for_analyzis = np.arange((window_beginning_ignore+ramp_duration)*fsamp,len(time)-(window_end_ignore+ramp_duration)*fsamp)
    else:
        samples_for_analyzis = np.arange(window_beginning_ignore*fsamp,len(time)-window_end_ignore*fsamp)
else:
    samples_for_analyzis = np.arange(window_beginning_ignore*fsamp,len(time)-window_end_ignore*fsamp)

In [ ]:
# Plot error improvement
fig, ax = plt.subplots(figsize=(10,6))
plt.plot(errors)
plt.xlabel('Iteration')
plt.ylabel('Cost (mean error in % of MVC)')
plt.ylim([0,np.ceil(max(errors))])
plt.title('Cost Improvement Over Iterations')
plt.grid(True)
new_filename = f'Optimization_of_input_loss.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Plot target vs output force (initial and final)
fig, ax = plt.subplots(figsize=(20,7))
plt.plot(target_force, label='Target Force')
plt.plot(initial_output_force, label='Initial Output Force')
if inhib_before_force_optimization:
    plt.plot(output_force, label='Final Output Force (optimization of common excitatory input taking inhibition into account)')
else:
    plt.plot(output_force, label='Final Output Force (optimization of common excitatory ignoring inhibition = so force should be lower)')
plt.xlabel('Time (ms)')
plt.ylabel('Force')
plt.legend()
plt.title('Force Output of Motor Neurons')
plt.grid(True)
new_filename = f'Optimized_input_resulting_force.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Plot control signal before and after learning
fig, ax = plt.subplots(figsize=(20,7))
plt.plot(initial_excit_signal, label='Initial Control Signal')
if inhib_before_force_optimization:
    optimized_signal_label = 'Optimized Control Signal'
    common_input_signal_label = 'Common input (optimized control signal + common noise)'
else:
    optimized_signal_label = 'Optimized Control Signal (ignoring inhibition which is applied later)'
    common_input_signal_label = 'Common input (optimized control signal ignoring inhibition + common noise)'
plt.plot(optimized_excit_signal, label=optimized_signal_label)
plt.plot(common_input, label=common_input_signal_label)
if nb_inhibitory_input >= 1:
    for inhibiti in range(nb_inhibitory_input):
        plt.plot(inhib_input[inhibiti][0], label=f'inhibitory input #{inhibiti+1}', color='C4', alpha = 1/nb_inhibitory_input)
plt.xlabel('Time (ms)')
plt.ylabel('Control Signal')
plt.legend()
# plt.ylim(0.5,2.5)
plt.title('Control Signal Before and After Learning + common input')
plt.grid(True)
new_filename = f'Optimized_input_&_common_input.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

print(f'Mean and std of common input during window of analyzis = {np.round(np.mean(common_input[samples_for_analyzis])*100)/100} +/- {np.round(np.std(common_input[samples_for_analyzis])*100)/100}')

# Getting a smooth color blend from a given colormap
colormap_temp = cm.get_cmap('plasma')

# Raster plot of discharge times
plt.figure(num=1,figsize=(20,10))
for mni in range(nb_motoneurons_full_pool):
    plt.scatter((spike_trains[mni]/second)*fsamp, np.ones(len(spike_trains[mni]))*mni, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), linewidth = 0.5, alpha = 0.2)
plt.vlines(samples_for_analyzis[0],plt.ylim()[0],plt.ylim()[1],color='black',label='Start of the analyzis window')
plt.vlines(samples_for_analyzis[len(samples_for_analyzis)-1],plt.ylim()[0],plt.ylim()[1],color='black',label='End of the analyzis window')
plt.xlabel('Time (s)')
plt.ylabel('Motoneuron index')
plt.title("Raster plot of motoneuron spikes")
plt.legend()
new_filename = f'Optimized_input_firings_raster_plot.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

# Force per MU
force_per_MU = []
force_total = zeros(len(binary_spike_trains[0,:]))
fig, ax = plt.subplots(figsize=(25, 10))

for mni in range(nb_motoneurons_full_pool):
    # 'same' mode means the output length will be the same as the input length
    if len(spike_trains[mni]) >= 1: # at least one spike necessary
        temp_force = np.convolve(binary_spike_trains[mni,:], twitch_convolution_window_normalized[mni], mode='same')
        force_per_MU.append(temp_force)
        ax.plot(force_total, color=colormap_temp(mni/(nb_motoneurons_full_pool-1)), alpha = 0.5, linewidth = 0.5)
        force_total = force_total + temp_force
if low_pass_filter_force:
    ax.plot(output_force, color='black', alpha = 1, linewidth = 2, label = "Total force (low-pass filtered)")
else:
    ax.plot(force_total, color='black', alpha = 1, linewidth = 2, label = "Total force")
plt.title(f"Reconstructed force (convolving spike train with twitch force kernel)")
plt.ylabel("Force (% MVC)")
plt.xlabel("Time (ms)")
plt.legend()
new_filename = f'Optimized_input_cumulative_force_per_MU.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)
 

In [ ]:
### Get discharge characteristics of motoneurons => only during window of analysis
fig, axs = plt.subplots()
# Retrieve spikes
spike_trains = []
for mni in range(nb_motoneurons_full_pool):
    spike_trains.append(monitor_spikes_motoneurons.spike_trains()[mni])
    # remove samples out of the analyzis window
    spike_trains[mni] = spike_trains[mni]/second # convert into a simple numpy array first
    spike_trains[mni] = spike_trains[mni][(spike_trains[mni] > (samples_for_analyzis[0]/fsamp)) & (spike_trains[mni] < (samples_for_analyzis[len(samples_for_analyzis)-1]/fsamp))]
    # spike_trains[mni] = spike_trains[mni]*second # convert back into a Brian2 array with unit
# Calculate the firing rate for each neuron
firing_rates = []
highest_ISIs = []
for mni in range(nb_motoneurons_full_pool):
    if len(spike_trains[mni]) <= 1:
        highest_ISIs.append(len(samples_for_analyzis)/fsamp)
    else:
        highest_ISIs.append(max(diff(spike_trains[mni])))
    firing_rate_temp = len(spike_trains[mni]) / (len(samples_for_analyzis)/fsamp)
    firing_rates.append(firing_rate_temp)
# Convert to a numpy array for easier calculations
firing_rates = np.array(firing_rates)
# Calculate mean and standard deviation of the firing rates
mean_firing_rate = np.mean(firing_rates)
std_firing_rate = np.std(firing_rates)
# Motoneurons' firing rates results
axs.hist(firing_rates, edgecolor='white', alpha=0.75)
axs.axvline(x = mean_firing_rate, linestyle='--', linewidth=2, label='Mean firing rate')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate (all motoneurons, only during window of analyzis)")

In [ ]:
### SELECT ONLY VALID (continuous) MOTONEURONS
discontinuous_MUs_idx = {}
valid_MUs_idx = {}
fig, axs = plt.subplots()

# Index of discontinuous MNs (ISIs > 0.4)
discontinuous_MUs_idx = [i for i, x in enumerate(highest_ISIs) if x > ISI_threshold_for_discontinuity]
discontinuous_MUs_idx = append(discontinuous_MUs_idx,
                            [i for i, x in enumerate(np.arange(nb_motoneurons_full_pool)) if len(spike_trains[x])<20]) # remove MUs with less than X spikes
discontinuous_MUs_idx = unique(discontinuous_MUs_idx)

valid_MUs_idx = [i for i, x in enumerate(arange(nb_motoneurons_full_pool)) if x not in discontinuous_MUs_idx]
print("Number of invalid MUs = ", len(discontinuous_MUs_idx), " out of ", nb_motoneurons_full_pool)
axs.hist(highest_ISIs, edgecolor='white', alpha=0.5)
axs.axvline(x = np.median(highest_ISIs), linestyle='--', linewidth=3, label='Mean highest ISI')
axs.axvline(x = ISI_threshold_for_discontinuity, color = 'black', linestyle='-', linewidth=2, alpha = 0.5, label='Mean firing rate')
axs.set_xlabel("Max ISI (s)")
axs.set_ylabel("Motoneuron count")
plt.tight_layout(rect=[0,0,1,0.92])
plt.suptitle("Histogram of motoneurons' max ISI \n (colored line is median ; black line is threshold)")
# new_filename = f'Hist_MN_ISIs.png'
# save_file_path = os.path.join(new_directory, new_filename)
# plt.savefig(save_file_path)
plt.show(fig)
    


In [ ]:
# Define the softmax function
def softmax_with_temperature(logits, temperature=1.0):
    """
    Compute the softmax of a list of logits with a temperature parameter.

    Parameters:
    logits (list or numpy array): The input logits.
    temperature (float): The temperature parameter.

    Returns:
    numpy array: The softmax probabilities.
    """
    # Convert logits to numpy array if they are not already
    logits = np.array(logits)
    
    # Apply the temperature parameter
    logits = logits / temperature
    
    # Compute the exponentials of the scaled logits
    exp_logits = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    
    # Compute the softmax probabilities
    softmax_probs = exp_logits / np.sum(exp_logits)
    
    return softmax_probs

In [ ]:
# SELECT A SUBSET OF MOTOR UNITS
plot_title_text = 'sampling of motor units for analysis' 

if motor_unit_subsampling_probability_distribution == 'uniform':
    sampling_probability_distribution = np.ones(shape(valid_MUs_idx))/len(valid_MUs_idx)
else:
    sampling_probability_distribution = np.copy(motoneuron_soma_diameters[valid_MUs_idx])
    sampling_probability_distribution = softmax_with_temperature(sampling_probability_distribution, bias_towards_larger_motor_neurons_temperature)

if subsample_MUs_for_analysis == False:
    selected_motor_units = valid_MUs_idx.copy()
    plot_title_text = plot_title_text + ' (all valid motor units selected)'
    txt_for_legend = f'selected motor units (n={len(valid_MUs_idx)}=all continuously active motor units)'
    plot_title_suffix = f' (all valid motor units)'
else:
    selected_motor_units = np.random.choice(valid_MUs_idx, size=nb_of_MUs_to_subsample, p=sampling_probability_distribution, replace=False)
    plot_title_text = plot_title_text + f' (subset of {nb_of_MUs_to_subsample} motor units selected)'
    txt_for_legend = f'selected motor units (n={nb_of_MUs_to_subsample})'
    plot_title_suffix = f' (sampling of {nb_of_MUs_to_subsample} motor units)'
# selected_motor_units_relative_to_valid_MUs = np.array([np.where(valid_MUs_idx == x)[0][0] for x in selected_motor_units])
selected_motor_units_relative_to_valid_MUs = [valid_MUs_idx.index(x) for x in selected_motor_units]

plt.figure(figsize=(20,5))
plt.bar(valid_MUs_idx,sampling_probability_distribution, label='probability of each valid (continuously firing) MU', color='C9', alpha=0.3)
plt.bar(selected_motor_units,sampling_probability_distribution[selected_motor_units_relative_to_valid_MUs], label=txt_for_legend, color='C9')
plt.legend()
plt.ylabel("Probability")
plt.xlabel("Motor unit index")
plt.title(plot_title_text)

new_filename = f'Valid_VS_sampled_motor_units_for_analyzis.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

In [ ]:
# Firing rate results - only selected MNs
fig, axs = plt.subplots()
axs.hist(firing_rates[selected_motor_units], edgecolor='white', alpha=0.75)
mean_firing_rate_valid = np.mean(firing_rates[selected_motor_units])
std_firing_rate_valid = np.std(firing_rates[selected_motor_units])
axs.axvline(x = mean_firing_rate_valid, linestyle='--', linewidth=2, label='Mean firing rate')
axs.set_xlabel("Mean firing rate (pps)")
axs.set_ylabel("Motoneuron count count")
plt.tight_layout(rect=[0,0,1,0.96])
plt.suptitle("Histogram of motoneurons' firing rate" + plot_title_suffix)
new_filename = f'Hist_MN_Discharge_rates.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)
    

In [ ]:
## SMOOTHING SPIKE TRAINS

Wind_s = 0.4  # hanning window duration. 0.4 for 2.5hz low-pass, 0.2 for 5hz low-pass
HanningW = 2 / round(fsamp * Wind_s) * windows.hann(round(fsamp * Wind_s))  # unitary area

# Filter all valid motor units
smoothed_signal = []
for mni in range(nb_motoneurons_full_pool):
    if mni in valid_MUs_idx:
        smoothed_signal.append(filtfilt(HanningW, 1, binary_spike_trains[mni, :] * fsamp))
smoothed_signal = np.array(smoothed_signal)

fig, ax = plt.subplots(figsize=(25, 10))
colormap_temp = cm.get_cmap('plasma') # Getting a smooth color blend from a given colormap
if subsample_MUs_for_analysis == True:
    for mni in range(len(valid_MUs_idx)):
        ax.plot((smoothed_signal)[mni,:], color=colormap_temp(valid_MUs_idx[mni]/(nb_motoneurons_full_pool-1)), alpha = 0.3)
# Remove non-valid motor units, and replot on top of the previous plot only the sampled motor units
smoothed_signal = smoothed_signal[selected_motor_units_relative_to_valid_MUs,:]
for mni in range(smoothed_signal.shape[0]):
    ax.plot((smoothed_signal)[mni,:], color=colormap_temp(selected_motor_units[mni]/(nb_motoneurons_full_pool-1)), alpha = 1)

plt.title(f"Smoothed signals of only continuous MUs \n (dark = small MNs ; light = large MNs) \n (low opacity = valid but not selected ; opaque = selected motor units)")
plt.ylabel("Smoothed discharge rate (pps)")
plt.xlabel("Time (ms)")
plt.vlines(samples_for_analyzis[0],plt.ylim()[0],plt.ylim()[1],color='black',label='Start of the analyzis window')
plt.vlines(samples_for_analyzis[len(samples_for_analyzis)-1],plt.ylim()[0],plt.ylim()[1],color='black',label='End of the analyzis window')
plt.legend()
new_filename = f'Smoothed_discharge_rates_only_continuous_MUs.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

# restrict signal to window of analyzis, with only valid motor units
smoothed_signal = smoothed_signal[:,samples_for_analyzis]
# re-plot
fig, ax = plt.subplots(figsize=(25, 10))
for mni in range(smoothed_signal.shape[0]):
    ax.plot(smoothed_signal[mni,:], color=colormap_temp(selected_motor_units[mni]/(nb_motoneurons_full_pool-1)))
plt.title(f"Smoothed signals only during window of analyzis" + plot_title_suffix + "\n (dark = small MNs ; light = large MNs)")
plt.ylabel("Smoothed discharge rate (pps)")
plt.xlabel("Time (ms)")
new_filename = f'Smoothed_discharge_rates_only_continuous_MUs_window_of_analyzis.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

    

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

# PCA
smoothed_signal_normalized = np.copy(smoothed_signal)
for mni in range(smoothed_signal.shape[0]):
    smoothed_signal_normalized[mni] = smoothed_signal_normalized[mni]-np.mean(smoothed_signal_normalized[mni])
    smoothed_signal_normalized[mni] = smoothed_signal_normalized[mni]/np.std(smoothed_signal_normalized[mni])

# Displaye smoothed signals normalized (mean of 0, std of 1)
fig, ax = plt.subplots(figsize=(25, 10))
for mni in range(smoothed_signal.shape[0]):
    ax.plot(smoothed_signal_normalized[mni,:], color=colormap_temp(selected_motor_units[mni]/(nb_motoneurons_full_pool-1)))
plt.title(f"Normalized smoothed signals of only continuous MUs, only during window of analyzis"  + plot_title_suffix + "\n (dark = small MNs ; light = large MNs)")
plt.ylabel("Normalizd smoothed discharge rate (in std)")
plt.xlabel("Time (ms)")
new_filename = f'Smoothed_discharge_rates_ormalized_only_continuous_MUs_window_of_analyzis.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show(fig)

nb_PCs_to_store = 5

# Function to calculate R-squared for each time series with a given number of PCs
def calculate_r_squared(data, pca, num_pcs):
    # Transform the data using the selected number of PCs
    transformed_data = pca.transform(data)
    # Inverse transform to reconstruct the data
    reconstructed_data = pca.inverse_transform(
        np.hstack([transformed_data, np.zeros((data.shape[0], pca.n_components_ - num_pcs))])
    )
    # Calculate R-squared for each time series (motor unit)
    r_squared_values = [r2_score(data[:, i], reconstructed_data[:, i]) for i in range(data.shape[1])]
    return r_squared_values

# Assume `smoothed_discharge_rates` is the variable holding     the data
# Perform PCA
nb_PCs = 10 # nb_motoneurons_full_pool
pca = PCA(n_components=nb_PCs)
pca_result = pca.fit_transform(transpose(smoothed_signal_normalized))

# Initialize a DataFrame to store the R-squared values
PCA_r_squared_df = pd.DataFrame(index=range(1, nb_PCs_to_store + 1), columns=range(smoothed_signal_normalized.shape[0])) # nb_PCs + 1
# Calculate R-squared values for 1 to 2 PCs
for num_pcs in range(1,nb_PCs_to_store+1): # From 1 to 2 PCs # nb_PCs + 1): 
    pca_temp = PCA(n_components=num_pcs)
    pca_temp.fit(smoothed_signal_normalized.T)
    r_squared_values = calculate_r_squared(smoothed_signal_normalized.T, pca_temp, num_pcs)
    PCA_r_squared_df.loc[num_pcs] = r_squared_values

# Display the explained variance ratio (proportion of variance explained by each PC)
explained_variance_ratio = pca.explained_variance_ratio_
# Optionally, display the cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Plot the explained variance
plt.figure(figsize=(8, 6))
plt.bar(range(0, nb_PCs + 1), append(0,explained_variance_ratio), alpha=0.5, align='center',
        label='Individual explained variance')
plt.plot(range(0, nb_PCs + 1), append(0,cumulative_explained_variance),
         label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.title('Explained Variance by Principal Components' + plot_title_suffix)
plt.ylim(-0.1,1.1)
plt.legend(loc='best')
plt.grid(True)
new_filename = f'Error_correction_PCA_VAF.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

print(f'Mean R² of PC1+PC2 = {np.mean(PCA_r_squared_df.iloc[1])}')
print(f'Cumulative VAF for PC1+PC2 = {cumulative_explained_variance[1]}')

In [ ]:
# Multivariate Fourier Analysis function - from NeuroSpec211 by Halliday - https://github.com/dmhalliday/NeuroSpec 
def sp2a2_R2(x, y, samp_rate, seg_pwr):

    # Check number of input arguments
    if x is None or y is None or samp_rate is None or seg_pwr is None:
        raise ValueError('Not enough input arguments')

    # Check for single column data
    if len(x.shape) != 1:
        raise ValueError('Input NOT single column: x')
    if len(y.shape) != 1:
        raise ValueError('Input NOT single column: y')

    pts_tot = len(x)  # Determine size of data vector
    if len(y) != pts_tot:  # Check that input vectors are equal length
        raise ValueError('Unequal length data arrays')

    if np.isscalar(samp_rate) is False:
        raise ValueError('Non scalar value for: samp_rate')
    if np.isscalar(seg_pwr) is False:
        raise ValueError('Non scalar value for: seg_pwr')

    # Segment for FFT
    seg_size = 2 ** seg_pwr  # Segment length, T
    seg_tot = pts_tot // seg_size  # No of segments, L
    samp_tot = seg_tot * seg_size  # Record length, R = LT

    x = x[:samp_tot].reshape((seg_size, seg_tot))  # Reshape x to T rows, L columns
    y = y[:samp_tot].reshape((seg_size, seg_tot))  # Reshape y to T rows, L columns

    # Create zero mean sequence for each segment
    x -= x.mean(axis=0)
    y -= y.mean(axis=0)

    # FFT & average periodogram
    dx = fft(x, axis=0)  # dFT across columns
    dy = fft(y, axis=0)

    psd_fac = 1 / (2 * np.pi * samp_tot)
    fxx = psd_fac * np.sum(np.abs(dx) ** 2, axis=1)
    fyy = psd_fac * np.sum(np.abs(dy) ** 2, axis=1)
    fyx = psd_fac * np.sum(dy * np.conj(dx), axis=1)

    # Coherence
    chyx = np.zeros(seg_size)
    chyx[1:] = np.abs(fyx[1:] ** 2) / (fxx[1:] * fyy[1:])

    # Pre-whitening stage - generate MMSE filters
    wx = np.zeros(seg_size)
    wy = np.zeros(seg_size)
    wx[1:] = 1.0 / np.sqrt(fxx[1:])
    wy[1:] = 1.0 / np.sqrt(fyy[1:])

    # Apply MMSE pre-whitening filters
    dxw = dx * wx[:, np.newaxis]
    dyw = dy * wy[:, np.newaxis]

    # Re-calculate spectra using pre-whitened processes
    fxxw = psd_fac * np.sum(np.abs(dxw) ** 2, axis=1)
    fyyw = psd_fac * np.sum(np.abs(dyw) ** 2, axis=1)
    fyxw = psd_fac * np.sum(dyw * np.conj(dxw), axis=1)

    chyxw = np.abs(fyxw) ** 2
    rhoyx = np.real(ifft(fyxw))

    # R2 in frequency domain
    R2_ch = np.sum(chyx) / seg_size
    R2_chw = np.sum(chyxw) / seg_size

    # Estimate R2 in time domain and separate components
    R2_rho = np.sum(rhoyx ** 2)
    R2_rho_0 = rhoyx[0] ** 2
    R2_rho_p = np.sum(rhoyx[1:seg_size // 2] ** 2)
    R2_rho_n = np.sum(rhoyx[seg_size // 2:] ** 2)

    # Decompose rho into three components by lag
    rhoyx_3 = np.zeros((seg_size, 3))
    rhoyx_3[0, 0] = rhoyx[0]
    rhoyx_3[1:seg_size // 2, 1] = rhoyx[1:seg_size // 2]
    rhoyx_3[seg_size // 2:, 2] = rhoyx[seg_size // 2:]

    # Switch back to frequency domain for f' estimates
    f_prime = fft(rhoyx_3, axis=0)
    f_prime2 = np.abs(f_prime) ** 2

    # Scale to get relative contributions
    f_prime2_fac = np.sum(f_prime2, axis=1)
    R2_weight = f_prime2 / f_prime2_fac[:, np.newaxis]

    R2_fprime_0 = np.sum(f_prime2[:, 0]) / seg_size
    R2_fprime_p = np.sum(f_prime2[:, 1]) / seg_size
    R2_fprime_n = np.sum(f_prime2[:, 2]) / seg_size

    # Construct output matrix f
    f = np.zeros((seg_size // 2, 12))
    f_index = np.arange(1, seg_size // 2 + 1)
    deltaf = samp_rate / seg_size

    f[:, 0] = (f_index - 1) * deltaf  # Column 1 - frequencies in Hz
    f[:, 1] = np.log10(fxx[f_index])  # Column 2 - Log spectrum ch 1, x
    f[:, 2] = np.log10(fyy[f_index])  # Column 3 - Log spectrum ch 2, y
    f[:, 3] = chyx[f_index]           # Column 4 - Coherence
    f[:, 4] = np.angle(fyx[f_index])  # Column 5 - Phase
    f[:, 5] = fxxw[f_index]           # Column 6 - MMSE whitened spectra process x
    f[:, 6] = fyyw[f_index]           # Column 7 - MMSE whitened spectra process y
    f[:, 7] = chyxw[f_index]          # Column 8 - Coherence between whitened processes
    f[:, 8] = np.angle(fyxw[f_index]) # Column 9 - Phase between whitened processes
    f[:, 9] = R2_weight[f_index, 0] * chyxw[f_index]  # Column 10 - Zero lag coherence component
    f[:, 10] = R2_weight[f_index, 1] * chyxw[f_index] # Column 11 - Forward coherence component
    f[:, 11] = R2_weight[f_index, 2] * chyxw[f_index] # Column 12 - Reverse coherence component

    # Construct time domain matrix t
    deltat = 1000.0 / samp_rate  # dt in msec
    t = np.zeros((seg_size, 3))
    t[:, 0] = (np.arange(1, seg_size + 1) - seg_size // 2 - 1) * deltat  # Lag range
    t[np.r_[seg_size // 2:seg_size, 0:seg_size // 2], 1] = 2 * np.pi * np.real(ifft(fyx))  # Cumulant density
    t[np.r_[seg_size // 2:seg_size, 0:seg_size // 2], 2] = rhoyx  # rhoyx

    # Estimate variance of cumulant density and rho estimates
    var_fac = 4 * np.pi ** 2 / (seg_size * samp_tot)
    q_var = var_fac * 2 * np.sum(fxx[:seg_size // 2 + 1] * fyy[:seg_size // 2 + 1])
    rho_var = 1 / samp_tot

    cl = {
        'type': 0,
        'seg_size': seg_size,
        'seg_tot': seg_tot,
        'seg_tot_var': seg_tot,
        'samp_tot': samp_tot,
        'samp_rate': samp_rate,
        'dt': deltat,
        'df': deltaf,
        'f_c95': 0.8512 * np.sqrt(1 / seg_tot),
        'ch_c95': 1 - 0.05 ** (1 / (seg_tot - 1)),
        'q_c95': 1.96 * np.sqrt(q_var),
        'rho_c95': 1.96 * np.sqrt(rho_var),
        'col_R20': 10,
        'col_R2p': 11,
        'col_R2n': 12,
        'col_rho': 3,
        'R2': R2_rho,
        'R2_0': R2_rho_0,
        'R2_p': R2_rho_p,
        'R2_n': R2_rho_n,
        'R2_ch': R2_ch,
        'R2_chw': R2_chw,
        'R2_fprime_0': R2_fprime_0,
        'R2_fprime_p': R2_fprime_p,
        'R2_fprime_n': R2_fprime_n,
        'N1': 0,
        'N2': 0,
        'P1': 0,
        'P2': 0,
        'opt_str': '',
        'what': ''
    }

    # print(f'Segments: {seg_tot}, Segment length: {seg_size/samp_rate} sec, Resolution: {cl["df"]} Hz.')
    # print(f'  R2 (0, p, n): {cl["R2"]} ({cl["R2_0"]}, {cl["R2_p"]}, {cl["R2_n"]}).')

    return f, t, cl

In [ ]:
from scipy.signal import csd, detrend

windowCOH = 1 # in seconds
frequencies_per_FFT_window = 10
indexes_of_windows_below_5hz = np.arange(0,5*frequencies_per_FFT_window)

# From groups of 2 up to half of the motor neurons
shuffled_idx_list = list.copy(list(selected_motor_units)) # initialize list of indices to be shuffled iteratively
window_analyzis_begin = int(samples_for_analyzis[0])
window_analyzis_end = int(samples_for_analyzis[len(samples_for_analyzis)-1])
seg_pwr = 10 # segment length (for coherence analyzis), specificied as a power of 2. 2^10 is ~1s for a fsamp of 1000

max_or_mean_0_5hz_COH = 'max'

COH_calc_group_size_nb = int(floor(len(selected_motor_units)/2)-1)
# COH_calc_max_iteration_nb_per_group_size = 1000 # More iteration for smaller group sizes, because the value obtained is very dependent upon the exact neurons selected, especially when only a few MNs are used to create the CST
COH_0_5hz_per_group = []
COH_mean_0_5hz = np.zeros(COH_calc_group_size_nb)
COH_pooled_per_group_size = []

colors_plots = plt.cm.winter(np.linspace(0, 1, COH_calc_group_size_nb))
plt.figure(figsize=(10,8))
for group_sizi in range(COH_calc_group_size_nb):
    coherence_temp = []
    COH_calc_iteration_nb_per_group_size_temp = int(np.max([np.round(COH_calc_max_iteration_nb_per_group_size/(group_sizi+1)),1]))
    COH_0_5hz_per_group.append([])
    print(f'Iterating for groups of {group_sizi+1} motoneurons (out of a max group size of {COH_calc_group_size_nb}) - {COH_calc_iteration_nb_per_group_size_temp} iterations')
    for group_iteri in range(COH_calc_iteration_nb_per_group_size_temp):
        random.shuffle(shuffled_idx_list)
        idx_cst1 = np.copy(shuffled_idx_list[0:group_sizi+1])
        cst1 = sum(binary_spike_trains[idx_cst1,window_analyzis_begin:window_analyzis_end],axis=0)
        idx_cst2 = np.copy(shuffled_idx_list[len(shuffled_idx_list)-(group_sizi+1):len(shuffled_idx_list)])
        cst2 = sum(binary_spike_trains[idx_cst2,window_analyzis_begin:window_analyzis_end],axis=0)

        # Compute intra-group coherence for group 1
        f, COH_intragroup_X = csd(detrend(cst1), detrend(cst1), window=windows.hann(round(windowCOH * fsamp)), noverlap=0, nfft=frequencies_per_FFT_window * fsamp, fs=fsamp)
        # Compute intra-group coherence for group 2
        f, COH_intragroup_Y = csd(detrend(cst2), detrend(cst2), window=windows.hann(round(windowCOH * fsamp)), noverlap=0, nfft=frequencies_per_FFT_window * fsamp, fs=fsamp)
        # Compute inter-group coherence
        f, COH_intergroup = csd(detrend(cst1), detrend(cst2), window=windows.hann(round(windowCOH * fsamp)), noverlap=0, nfft=frequencies_per_FFT_window * fsamp, fs=fsamp)

        coherence_temp.append( (np.abs(COH_intergroup) ** 2) / (COH_intragroup_X * COH_intragroup_Y) ) # Welch's method of coherence calculation
        if max_or_mean_0_5hz_COH == 'mean':
            # COH_0_5hz_per_group[group_sizi,group_iteri] = np.nanmean(coherence_temp[group_iteri][indexes_of_windows_below_5hz])
            COH_0_5hz_per_group[group_sizi].append(np.nanmean(coherence_temp[group_iteri][indexes_of_windows_below_5hz]))
        elif max_or_mean_0_5hz_COH == 'max':
            # COH_0_5hz_per_group[group_sizi,group_iteri] = np.nanmax(coherence_temp[group_iteri][indexes_of_windows_below_5hz])
            COH_0_5hz_per_group[group_sizi].append(np.nanmax(coherence_temp[group_iteri][indexes_of_windows_below_5hz]))

        # plt.scatter(group_sizi,COH_0_5hz_per_group[group_sizi,group_iteri],s=30,color=colors_plots[group_sizi],alpha=min(3/COH_calc_iteration_nb_per_group_size,1))
        plt.scatter(group_sizi,COH_0_5hz_per_group[group_sizi][group_iteri],s=30,color=colors_plots[group_sizi],alpha=min(np.sqrt(1/COH_calc_iteration_nb_per_group_size_temp),1))
    COH_pooled_per_group_size.append( np.nanmean(coherence_temp, axis=0) )
    # COH_mean_0_5hz[group_sizi] = np.nanmean(COH_0_5hz_per_group[group_sizi,:],axis=0)
    COH_mean_0_5hz[group_sizi] = np.nanmean(COH_0_5hz_per_group[group_sizi],axis=0)

plt.plot(COH_mean_0_5hz, linewidth=3, color='red', alpha=0.5, label=f"Mean of [max 0-5hz coherence for CSTs of X spike trains]")
plt.xlabel('Number of MNs in the CSTs')
plt.ylabel('Mean coherence in the 0-5hz bandwidth')
plt.title("Increase in coherence in the 0-5hz bandwidth (Y) as the number of CSTS' MNs (X) increase" + plot_title_suffix)
plt.ylim(0,1)
plt.legend()
new_filename = f'PCI_curve_of_0-5hz_coh.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

plt.figure(figsize=(10, 8))
for group_sizi in range(COH_calc_group_size_nb):
    plt.plot(COH_pooled_per_group_size[group_sizi], color=colors_plots[group_sizi], alpha = 0.5, linewidth = 2)
plt.xlim(0,20*frequencies_per_FFT_window)
plt.xticks(ticks=plt.xticks()[0], labels=[str(int(x / frequencies_per_FFT_window)) for x in plt.xticks()[0]])
plt.xlabel("Frequency (Hz)")
plt.ylabel("Coherence")
plt.ylim(0,1)
plt.title("Mean coherence" + plot_title_suffix)
new_filename = f'Coherence_curve.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

In [ ]:
# root mean square error function definition
def rmse(predicted, true):
    # Root mean square
    # return np.sqrt(np.mean((true - predicted) ** 2))
    # Mean square
    return np.mean((true - predicted) ** 2)

In [ ]:
from scipy.optimize import curve_fit, least_squares

# Define the model - implementation from Negro et al 2016 https://physoc.onlinelibrary.wiley.com/doi/epdf/10.1113/JP271748 
def PCI_model(n, A, B):
    return abs(n**2 * A)**2 / ((n*B) + ((n**2)*A))**2
# Define the residuals function
def residuals(params, x, y):
    A, B = params
    return y - PCI_model(x, A, B)

plt.figure(figsize=(12,8))
n = np.arange(1,COH_mean_0_5hz.shape[0]+1)
# Negro 2016 equation:
# Mean COH in a given frequency band: Eqn 4 = abs(n**2 * A)**2 / ((n*B) + ((n**2)*A))**2
#   - n = number of neurons in the CST
#   - A = power of the common synaptic input (in the given frequency band), multipled by the absolute square of the susceptibility
#   - A = abs(X(f))**2 * Ss(f)
#       => X(f) is the response function (susceptibility) of the motor neuron [POOl of motor neurons in our case] to the stimulus and Ss(f) the power spectrum of the stimulus
#   - B = (power of the?) response of the pool with independent synaptic input
#   - B = Sn(f)
#       => Sn(f)is the power spectrum of the output spike train [CST of spike train in our case] when it is driven by independent synaptic input only
#   - proportion of common input (PCI) = estimate of gamma (common voltage fluctuation of membrane) = sqrt(A/(B+A)) = proportion of the common synaptic input with respect to the total synaptic input received by the motor neurons [total input can be inferred from motor pool output, because it the output is a function of both the common and independent input]
#       => They say just sqrt(A/B) in the paper, but this can result in a proportion (PCI) > 1 which shouldn't be possible, and I get results fitting very closely the theoritical values when computing sqrt(A_fit / (B_fit + A_fit))
#   - The ratio can be estimated by an experimental measure of the mean coherence in the given frequency range for varying n (number of motor neuron spike trains used in the calculation (Negro & Farina, 2012)) using eqn (4)
#   - Using a least-square curve fitting of the estimated values of coherence for CSTs with different numbers of motor neurons, the parameters A and B of eqn (4) can be estimated.

# params, covariance = curve_fit(PCI_model, xdata = n, ydata = COH_mean_0_5hz) # older version, still works well
initial_guess = [1, 1] # Initial guess for the parameters from which to optimize using least squares
least_square_optim_results = least_squares(residuals, initial_guess, loss='soft_l1', f_scale=0.1, args=(n, COH_mean_0_5hz)) # default loss function is 'linear' but 'soft_l1' is a bit more robust to fluctuations
# Extract parameters
A_fit, B_fit = least_square_optim_results.x
PCI_estimated = np.sqrt(A_fit/(B_fit+A_fit))
fitted_PCI_curve = PCI_model(n, A_fit, B_fit)

# Get ratio of common excitatory input to independent input = Grond-truth PCI
# = power spectrum integral of common input in the 0-5hz range over power spectrum integral of independent input in the same range
# Ignoring inhibitory input = hard to calculate, so no ground truth when simulating inhibition
PCI_ground_truth_without_inhib = np.sqrt(common_noise_power_intergal_0_5_hz / (common_noise_power_intergal_0_5_hz + independent_input_noise_power_integral_0_5_hz_mean))

plt.plot(COH_mean_0_5hz, color='C0',linewidth=2.5, alpha = 0.5, label='ground truth curve')
plt.plot(fitted_PCI_curve, color='blue',linewidth=2, alpha = 0.5, linestyle='dashed',label='fitted curve')
plt.xlabel('Number of MNs in the CSTs')
plt.ylabel('Mean coherence in the 0-5hz bandwidth')
plt.title(f"Increase in coherence in the 0-5hz bandwidth (Y) as the number of CSTS' MNs (X) increase => Ground-truth VS fitted data" + plot_title_suffix + f"\n Estimated PCI (gamma) = {np.round(PCI_estimated*1000)/1000} \n Ground-truth PCI (ratio of common input 0-5hz integral of power spectrum VS idem common+independent input) = {np.round(PCI_ground_truth_without_inhib*1000)/10}%, ignoring inhibition \n")
plt.ylim(0,1)
plt.legend()
new_filename = f'PCI_fitted_vs_true_curve.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()


In [ ]:
# # SIMON'S INSPIRED VERSION - Implementation from Negro et al 2016 https://physoc.onlinelibrary.wiley.com/doi/epdf/10.1113/JP271748 
# plt.figure(figsize=(12,8))
# n = np.arange(1,COH_mean_0_5hz.shape[0]+1)
# # Negro 2016 equation:
# # Mean COH in a given frequency band: Eqn 4 = (abs(n**2 * A)**2) / ((n*B) + (((n**2)*A))**2)
# #   - n = number of neurons in the CST
# #   - A = power of the common synaptic input (in the given frequency band), multipled by the absolute square of the susceptibility (which is 1 when considering the 0-5hz bandwidth I think)
# #   - B = response of the pool with independent synaptic input (of the same bandwidth). Assumed to be 1 for the 0-5hz bandwidth.
# #       (k in the fitting below correspond to sqrt(B), since)
# #   #   #
# #   - proportion of common input (PCI) = estimate of gamma (common voltage fluctuation of membrane) = sqrt(A/B) = proportion of the common synaptic input with respect to the total synaptic input received by the motor neurons
# #   - The ratio can be estimated by an experimental measure of the mean coherence in the given frequency range for varying n (number of motor neuron spike trains used in the calculation (Negro & Farina, 2012)) using eqn (4)
# #   - Using a least-square curve fitting of the estimated values of coherence for CSTs with different numbers of motor neurons, the parameters A and B of eqn (4) can be estimated.
# nb_gammas_to_try = int(1e3) # Correspond to B
# gamma_tmp = np.linspace(0.1,1e2,nb_gammas_to_try)
# gamma_fit_error = []
# for k in gamma_tmp:
#     c = n**4 / (n / k**2 + n**2)**2
#     plt.plot(c, color='C0',linewidth=1, alpha = min(5/nb_gammas_to_try,1))
#     gamma_fit_error.append(rmse(c,COH_mean_0_5hz))
# idx_with_min_error = gamma_fit_error.index(min(gamma_fit_error))
# gamma = np.sqrt(gamma_tmp[idx_with_min_error]**2 / (gamma_tmp[idx_with_min_error]**2 +1))
# fitted_PCI_curve = n**4 / ( n / gamma_tmp[idx_with_min_error]**2 + n**2 )**2
# # fitted_PCI_curve = n**4 / ( n / gamma**2 + n**2 )**2

# # Get ratio of common excitatory input to independent input = Grond-truth PCI
# # = power spectrum integral of common input in the 0-5hz range over power spectrum integral of independent input in the same range
# # Ignoring inhibitory input = hard to calculate, so no ground truth when simulating inhibition
# PCI_ground_truth_without_inhib = np.sqrt(common_noise_power_intergal_0_5_hz / (common_noise_power_intergal_0_5_hz + independent_input_noise_power_integral_0_5_hz_mean))

# plt.plot(COH_mean_0_5hz, color='C0',linewidth=2.5, alpha = 0.5, label='ground truth curve')
# plt.plot(fitted_PCI_curve, color='blue',linewidth=2, alpha = 0.5, linestyle='dashed',label='fitted curve')
# plt.xlabel('Number of MNs in the CSTs')
# plt.ylabel('Mean coherence in the 0-5hz bandwidth')
# plt.title(f"Increase in coherence in the 0-5hz bandwidth (Y) as the number of CSTS' MNs (X) increase => Ground-truth VS fitted data \n Estimated PCI (gamma) = {np.round(gamma*1000)/1000} \n Ground-truth PCI (ratio of common input variance VS independent input variance) = {np.round(PCI_ground_truth_without_inhib*1000)/10}%, ignoring inhibition \n")
# plt.ylim(0,1)
# plt.legend()
# new_filename = f'PCI_fitted_vs_true_curve.png'
# save_file_path = os.path.join(new_directory, new_filename)
# plt.savefig(save_file_path)
# plt.show()


In [ ]:
# Get recruitment threshold and discharge rates of motor units = only if chosing the "trapezoid" force target (because allows for gradual recruitment of motor units)
if target_type == 'trapezoid':
    MN_mean_firing_rates = np.copy(firing_rates) # all motoneurons by index, regardless of whether they are valid or not
    # Discharge rates duging the window of analyzis (the plateau section of the trapezoid)
    MN_recruitment_thresholds = np.full(firing_rates.shape, np.nan) # in % MVC
    spike_trains_full, binary_spike_trains_full = Get_binary_spike_trains(monitor_spikes_motoneurons, duration_with_ignored_window)
    for mni in valid_MUs_idx:
        # RT as force during the median time of the first 3 firings
        temp_RT = (spike_trains_full[mni]/second)*fsamp
        temp_RT = np.median(temp_RT[0:2])
        MN_recruitment_thresholds[mni] = output_force[int(np.round(temp_RT))]

MN_recruitment_thresholds_only_selected_idx = np.copy(MN_recruitment_thresholds)
MN_recruitment_thresholds_only_selected_idx = MN_recruitment_thresholds_only_selected_idx[selected_motor_units].reshape(-1, 1)
MN_mean_firing_rates_only_selected_idx = np.copy(MN_mean_firing_rates)
MN_mean_firing_rates_only_selected_idx = MN_mean_firing_rates[selected_motor_units].reshape(-1, 1)

# histogram of recruitment thresholds
plt.figure()
plt.hist(MN_recruitment_thresholds_only_selected_idx, density=True,
         edgecolor='white', alpha=0.75, color='C1')
plt.xlim(0,target_force_level*1.1)
ylabel("Proportion")
xlabel("Recruitment threshold (% MVC)")
title("Histogram of recruitment tresholds" + plot_title_suffix)
new_filename = f'RT_hsitogram.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

true_false_continuously_firing_MUs = np.zeros(nb_motoneurons_full_pool)
true_false_continuously_firing_MUs[valid_MUs_idx] = 1
true_false_sampled_motor_unit = np.zeros(nb_motoneurons_full_pool)
true_false_sampled_motor_unit[selected_motor_units] = 1

if nb_inhibitory_input == 0:
    MN_inhibition_weights = [np.zeros(shape(MN_inhibition_weights))]

sampling_probability_dsitribution_to_save = np.zeros(nb_motoneurons_full_pool)
if subsample_MUs_for_analysis == True:
    sampling_probability_dsitribution_to_save[valid_MUs_idx] = sampling_probability_distribution
else:
    sampling_probability_dsitribution_to_save[valid_MUs_idx] = 1

# Save recruitment thresholds and firing rates
RT_mean_DR_dataframe = pd.DataFrame({
    'MU_idx': np.arange(nb_motoneurons_full_pool),
    'Continusouly_firing': true_false_continuously_firing_MUs,
    'Sampled_for_analyzis': true_false_sampled_motor_unit,
    'Recruitment_threshold': MN_recruitment_thresholds,
    'Mean_firing_rate': MN_mean_firing_rates,
    'Soma_size': motoneuron_soma_diameters,
    'Probability_of_being_sampled': sampling_probability_dsitribution_to_save,
    'Inhibition_weight_(1st_inhibitory_input)': MN_inhibition_weights[0]
})
new_filename = f'Individual_MUs_results.csv'
save_file_path = os.path.join(new_directory, new_filename)
RT_mean_DR_dataframe.to_csv(save_file_path, index=False)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Create a Linear Regression model
model = LinearRegression()
# Fit the model
model.fit(MN_recruitment_thresholds_only_selected_idx, MN_mean_firing_rates_only_selected_idx)
# Get the slope (coefficient)
slope = model.coef_[0]
# Predict the Y values using the fitted model
Y_pred = model.predict(MN_recruitment_thresholds_only_selected_idx)
# Calculate R squared
r_squared = r2_score(MN_mean_firing_rates_only_selected_idx, Y_pred)

plt.figure()
plt.scatter(MN_recruitment_thresholds_only_selected_idx,MN_mean_firing_rates_only_selected_idx,s=70,alpha=0.35,color='C1')
plt.plot(MN_recruitment_thresholds_only_selected_idx, Y_pred,color='C3',alpha=0.5, linewidth=3, label=f'Linear regression (slope = {np.round(slope[0]*100)/100}; R² = {np.round(r_squared*100)/100})')
plt.xlabel('Recruitment threshold (% MVC)')
plt.ylabel("Mean firing rate on the plateau (pps)")
plt.ylim(0,np.ceil(max(MN_mean_firing_rates)/10)*10)
plt.title("Recruitment threshold to mean discharge rate relationship" + plot_title_suffix)
plt.legend()
plt.xlim(0,target_force_level*1.1)
new_filename = f'RT_to_mean_DR_relationship.png'
save_file_path = os.path.join(new_directory, new_filename)
plt.savefig(save_file_path)
plt.show()

mean_inhib_weights_of_active_MUs = 0
if nb_inhibitory_input > 0:
    for inhibiti in range(nb_inhibitory_input):
        mean_inhib_weights_of_active_MUs += np.mean(MN_inhibition_weights[inhibiti][selected_motor_units])
    mean_inhib_weights_of_active_MUs = mean_inhib_weights_of_active_MUs / nb_inhibitory_input

### Force output
force_output_window_of_analyzis = output_force[samples_for_analyzis]
force_output_error_window_of_analyzis = np.abs(output_force[samples_for_analyzis] - target_force[samples_for_analyzis])
mean_force_error_window_of_analyzis = np.mean(force_output_error_window_of_analyzis)
mean_force_output_window_of_analyzis = np.mean(force_output_window_of_analyzis)
std_force_output_window_of_analyzis = np.std(force_output_window_of_analyzis)


### Main results
main_results_dataframe = pd.DataFrame({
    'Simulation_name': sim_name,
    'Window_of_analyzis_duration': (window_analyzis_end-window_analyzis_begin)/fsamp,
    'Ground_truth_PCI_ignoring_inhibition': [PCI_ground_truth_without_inhib], #[] to make sure at least the first value is an array
    'PCSI_estimation': PCI_estimated,
    'VAF_for_PC1': cumulative_explained_variance[0],
    'Slope_of_RT_mean_DR_relationship': slope[0],
    'Nb_of_continously_active_MUs': len(valid_MUs_idx),
    'Nb_of_sampled_MUs_for_analyzis': len(selected_motor_units),
    'Mean_DR': np.mean(MN_mean_firing_rates_only_selected_idx),
    'std_DR': np.std(MN_mean_firing_rates_only_selected_idx),
    'Mean_RT': np.mean(MN_recruitment_thresholds_only_selected_idx),
    'std_RT': np.std(MN_recruitment_thresholds_only_selected_idx),
    '0-5hz power of common input': common_noise_power_intergal_0_5_hz,
    '0-5hz power of inhibition': inhibitory_input_power_integral_0_5_hz_mean,
    '0-5hz power of independent noise': independent_input_noise_power_integral_0_5_hz_mean,
    'Mean weight of inhibition': mean_inhib_weights_of_active_MUs,
    'Mean torque': mean_force_output_window_of_analyzis,
    'Mean torque error': mean_force_error_window_of_analyzis,
    'Torque variability (torque std)': std_force_output_window_of_analyzis
    })
new_filename = f'Main_results.csv'
save_file_path = os.path.join(new_directory, new_filename)
main_results_dataframe.to_csv(save_file_path, index=False)

### Cumulative Explained Variance
VAF_cumsum_dataframe = pd.DataFrame({
    'PC': np.arange(len(cumulative_explained_variance))+1,
    'Cumulative_VAF': cumulative_explained_variance
    })
new_filename = f'Cumulative_VAF_PCA.csv'
save_file_path = os.path.join(new_directory, new_filename)
VAF_cumsum_dataframe.to_csv(save_file_path, index=False)